In [1]:
import pandas as pd
import os
import json
import tiktoken
import openai

from time import sleep

pd.set_option('display.max_columns', 100)

In [2]:
SLEEP_TIME = 2
TEMPERATURE = 0.5
MAX_TOKENS = 3
PICKLE_PATH = '../data/pickle/'
PREDICTIONS_PATH = PICKLE_PATH + 'predictions.pkl'

PRIV1_KEY = 'anon_string1'
PRIV2_KEY = 'anon_string2'

UNRECOGNIZED_MODE_EXCEPTION = Exception( "Unrecognized operation mode" )

SYSTEM_DIRECTIVE_PROMPT_TUNING_0_EXAMPLE = { 
    "role": "system", 
    "content": """Your task is to use various attributes of the traffic transmitted/received by different IoT devices to classify them into one of 9 specific device types. You'll be provided a comma-separated tuple with 30 traffic attributes in the following order:

Communication Attributes

1. src_oui (e.g., 'Cisco'): Manufacturer of the device sending the frame.
2. dst_oui (e.g., 'Samsung' or 'multicast'): The destination that the frame is headed for.
3. protocol (e.g., 58 for IPV6 Multicast): Protocol number inside the Layer 2 Frame.

Transmission Attributes (Time in milliseconds, Size in bytes, Packets in count)

4. bidirectional_total_duration_ms: Total duration of bidirectional communication between the source and destination.
5. bidirectional_total_packets: Total number of packets transmitted bidirectionally across the source-destination channel.
6. bidirectional_total_bytes: Total bytes transmitted bidirectionally across the source-destination channel.
7. src2dst_total_duration_ms: Total duration of unidirectional communication from source to destination.
8. src2dst_total_packets: Total number of packets transmitted from source to destination.
9. src2dst_total_bytes: Total bytes transmitted from source to destination.
10. dst2src_total_duration_ms: Total duration of unidirectional communication from destination to source.
11. dst2src_total_packets: Total number of packets transmitted from destination to source.
12. dst2src_total_bytes: Total bytes transmitted from destination to source.

Packet Size Attributes (Size in bytes)

13. bidirectional_min_ps: Smallest packet size across bidirectional communication.
14. bidirectional_mean_ps: Mean packet size across bidirectional communication.
15. bidirectional_stdev_ps: Standard deviation in packet size across bidirectional communication.
16. bidirectional_max_ps: Largest packet size across bidirectional communication.
17. src2dst_min_ps: Smallest packet size from source to destination.
18. src2dst_mean_ps: Mean packet size from source to destination.
19. src2dst_stdev_ps: Standard deviation in packet size from source to destination.
20. src2dst_max_ps: Largest packet size from source to destination.
21. dst2src_min_ps: Smallest packet size from destination to source.
22. dst2src_mean_ps: Mean packet size from destination to source.
23. dst2src_stdev_ps: Standard deviation in packet size from destination to source.
24. dst2src_max_ps: Largest packet size from destination to source.

Transmission Rate Attributes (Bytes per millisecond, Packets per millisecond)

25. bidirectional_transmission_rate_byte_ms: Number of bytes transmitted per millisecond bidirectionally.
26. bidirectional_transmission_rate_ms: Number of packets transmitted per millisecond bidirectionally.
27. dst2src_transmission_rate_bytes_ms: Number of bytes transmitted per millisecond from destination to source.
28. dst2src_transmission_rate_ms: Number of packets transmitted per millisecond from destination to source.
29. src2dst_transmission_rate_bytes_ms: Number of bytes transmitted per millisecond from source to destination.
30. src2dst_transmission_rate_ms: Number of packets transmitted per millisecond from source to destination.

You can only classify the traffic into only one of the following 9 IoT device types, exactly as they appear below:

camera
router
mobile
switch
speaker
bulb
computer
motion_sensor
printer

Note: Not all attributes might be available for every frame, and some may be more telling than others in identifying the device type.""" 
}

In [3]:
SYSTEM_DIRECTIVE_PROMPT_TUNING_3_EXAMPLE = { 
    "role": "system", 
    "content": """Your task is to use various attributes of the traffic transmitted/received by different IoT devices to classify them into one of 9 specific device types. You'll be provided a comma-separated tuple with 30 traffic attributes in the following order:

Communication Attributes

1. src_oui: Manufacturer of the device sending the frame.
2. dst_oui: The destination that the frame is headed for.
3. protocol: Protocol number inside the Layer 2 Frame.

Transmission Attributes (Time in milliseconds, Size in bytes, Packets in count)

4. bidirectional_total_duration_ms: Total duration of bidirectional communication between the source and destination.
5. bidirectional_total_packets: Total number of packets transmitted bidirectionally across the source-destination channel.
6. bidirectional_total_bytes: Total bytes transmitted bidirectionally across the source-destination channel.
7. src2dst_total_duration_ms: Total duration of unidirectional communication from source to destination.
8. src2dst_total_packets: Total number of packets transmitted from source to destination.
9. src2dst_total_bytes: Total bytes transmitted from source to destination.
10. dst2src_total_duration_ms: Total duration of unidirectional communication from destination to source.
11. dst2src_total_packets: Total number of packets transmitted from destination to source.
12. dst2src_total_bytes: Total bytes transmitted from destination to source.

Packet Size Attributes (Size in bytes)

13. bidirectional_min_ps: Smallest packet size across bidirectional communication.
14. bidirectional_mean_ps: Mean packet size across bidirectional communication.
15. bidirectional_stdev_ps: Standard deviation in packet size across bidirectional communication.
16. bidirectional_max_ps: Largest packet size across bidirectional communication.
17. src2dst_min_ps: Smallest packet size from source to destination.
18. src2dst_mean_ps: Mean packet size from source to destination.
19. src2dst_stdev_ps: Standard deviation in packet size from source to destination.
20. src2dst_max_ps: Largest packet size from source to destination.
21. dst2src_min_ps: Smallest packet size from destination to source.
22. dst2src_mean_ps: Mean packet size from destination to source.
23. dst2src_stdev_ps: Standard deviation in packet size from destination to source.
24. dst2src_max_ps: Largest packet size from destination to source.

Transmission Rate Attributes (Bytes per millisecond, Packets per millisecond)

25. bidirectional_transmission_rate_byte_ms: Number of bytes transmitted per millisecond bidirectionally.
26. bidirectional_transmission_rate_ms: Number of packets transmitted per millisecond bidirectionally.
27. dst2src_transmission_rate_bytes_ms: Number of bytes transmitted per millisecond from destination to source.
28. dst2src_transmission_rate_ms: Number of packets transmitted per millisecond from destination to source.
29. src2dst_transmission_rate_bytes_ms: Number of bytes transmitted per millisecond from source to destination.
30. src2dst_transmission_rate_ms: Number of packets transmitted per millisecond from source to destination.

You can only classify the traffic into only one of the following 9 IoT device types, exactly as they appear below:

camera
router
mobile
switch
speaker
bulb
computer
motion_sensor
printer

Note: Not all attributes might be available for every frame, and some may be more telling than others in identifying the device type.

Now, you will be given 3 examples of tuples for each type of device. The order of the attributes in the tuples will be same as the order in which they were defined above.

Examples for camera:
1. (Insteon,Multicast,2,112030.0,4.0,240.0,112030.0,4.0,240.0,1.0,0.0,0.0,60.0,60.0,0.0,60.0,60.0,60.0,0.0,60.0,0.0,0.0,0.0,0.0,0.0021422833169686692,3.5704721949477816e-05,0.0,0.0,0.0021422833169686692,3.5704721949477816e-05)
2. (Insteon,TP-LINK TECHNOLOGIES CO.LTD.,17,216.0,2.0,340.0,1.0,1.0,78.0,1.0,1.0,262.0,78.0,170.0,130.10764773832474,262.0,78.0,78.0,0.0,78.0,262.0,262.0,0.0,262.0,1.5740740740740742,0.009259259259259259,262.0,1.0,78.0,1.0)
3. (Samsung Electronics Co.Ltd,Multicast,2,1.0,1.0,46.0,1.0,1.0,46.0,1.0,0.0,0.0,46.0,46.0,0.0,46.0,46.0,46.0,0.0,46.0,0.0,0.0,0.0,0.0,46.0,1.0,0.0,0.0,46.0,1.0)

Examples for router:
1. (TP-LINK TECHNOLOGIES CO.LTD.,Invoxia,6,1740517.0,102.0,15258.0,1740477.0,42.0,6924.0,1740384.0,60.0,8334.0,66.0,149.5882352941177,227.36559964037056,785.0,66.0,164.85714285714292,243.02058242939128,753.0,66.0,138.90000000000003,217.19090251572047,785.0,0.008766360799693425,5.8603277072272204e-05,0.0047885983782889295,3.447515031165536e-05,0.003978219763892312,2.4131315725516626e-05)
2. (TP-LINK TECHNOLOGIES CO.LTD.,Samsung Electronics Co.Ltd,6,696.0,4.0,318.0,696.0,2.0,155.0,8.0,2.0,163.0,66.0,79.5,15.92691642053372,97.0,66.0,77.5,16.263455967290593,89.0,66.0,81.5,21.920310216782973,97.0,0.45689655172413796,0.005747126436781609,20.375,0.25,0.22270114942528735,0.0028735632183908046)
3. (TP-LINK TECHNOLOGIES CO.LTD.,Netatmo,17,8117.0,4.0,1376.0,81.0,3.0,1026.0,1.0,1.0,350.0,342.0,344.0,4.0,350.0,342.0,342.0,0.0,342.0,350.0,350.0,0.0,350.0,0.16952075890107182,0.0004927929037821855,350.0,1.0,12.666666666666666,0.037037037037037035)

Examples for mobile:
1. (Samsung Electronics Co.Ltd,TP-LINK TECHNOLOGIES CO.LTD.,6,1.0,1.0,97.0,1.0,1.0,97.0,1.0,0.0,0.0,97.0,97.0,0.0,97.0,97.0,97.0,0.0,97.0,0.0,0.0,0.0,0.0,97.0,1.0,0.0,0.0,97.0,1.0)
2. (Samsung Electronics Co.Ltd,TP-LINK TECHNOLOGIES CO.LTD.,58,5770.0,4.0,336.0,5770.0,2.0,172.0,5009.0,2.0,164.0,78.0,84.0,4.000000000000001,86.0,86.0,86.0,0.0,86.0,78.0,82.0,5.656854249492381,86.0,0.058232235701906415,0.0006932409012131716,0.0327410660810541,0.0003992812936713915,0.02980935875216638,0.0003466204506065858)
3. (Samsung Electronics Co.Ltd,TP-LINK TECHNOLOGIES CO.LTD.,6,143.0,4.0,314.0,143.0,2.0,157.0,1.0,2.0,157.0,66.0,78.5,14.433756729740644,91.0,66.0,78.5,17.67766952966369,91.0,66.0,78.5,17.67766952966369,91.0,2.195804195804196,0.027972027972027972,157.0,2.0,1.097902097902098,0.013986013986013986)

Examples for switch:
1. (Belkin International Inc.,TP-LINK TECHNOLOGIES CO.LTD.,6,219.0,3.0,426.0,219.0,2.0,280.0,1.0,1.0,146.0,66.0,142.0,74.08103670980854,214.0,66.0,140.0,104.65180361560904,214.0,146.0,146.0,0.0,146.0,1.9452054794520548,0.0136986301369863,146.0,1.0,1.278538812785388,0.0091324200913242)
2. (Belkin International Inc.,TP-LINK TECHNOLOGIES CO.LTD.,6,217.0,3.0,426.0,217.0,2.0,280.0,1.0,1.0,146.0,66.0,142.0,74.08103670980854,214.0,66.0,140.0,104.65180361560904,214.0,146.0,146.0,0.0,146.0,1.9631336405529953,0.013824884792626729,146.0,1.0,1.2903225806451613,0.009216589861751152)
3. (Belkin International Inc.,Belkin International Inc.,6,226.0,13.0,1386.0,226.0,7.0,789.0,219.0,6.0,597.0,66.0,106.61538461538464,75.45698384017486,259.0,66.0,112.71428571428572,78.7204215048723,253.0,66.0,99.5,78.20421983499355,259.0,6.132743362831858,0.05752212389380531,2.7260273972602738,0.0273972602739726,3.4911504424778763,0.030973451327433628)

Examples for speaker:
1. (Amazon Technologies Inc.,TP-LINK TECHNOLOGIES CO.LTD.,6,1995.0,20.0,7599.0,1995.0,11.0,1469.0,1742.0,9.0,6130.0,66.0,379.9500000000001,507.56420128004754,1484.0,66.0,133.54545454545456,155.40486712864796,583.0,66.0,681.1111111111111,629.3996036788641,1484.0,3.8090225563909774,0.010025062656641603,3.51894374282434,0.005166475315729047,0.7363408521303259,0.005513784461152882)
2. (Amazon Technologies Inc.,TP-LINK TECHNOLOGIES CO.LTD.,6,147372.0,28.0,2187.0,147372.0,15.0,1046.0,147163.0,13.0,1141.0,54.0,78.10714285714286,40.216395482545,257.0,54.0,69.73333333333333,20.09430149024724,95.0,54.0,87.76923076923076,54.58197786533855,257.0,0.014839996742936243,0.00018999538582634422,0.007753307556926673,8.833742177041783e-05,0.007097684770512716,0.00010178324240697012)
3. (Amazon Technologies Inc.,TP-LINK TECHNOLOGIES CO.LTD.,17,191.0,2.0,180.0,1.0,1.0,90.0,1.0,1.0,90.0,90.0,90.0,0.0,90.0,90.0,90.0,0.0,90.0,90.0,90.0,0.0,90.0,0.9424083769633508,0.010471204188481676,90.0,1.0,90.0,1.0)

Examples for bulb:
1. (LIFI LABS MANAGEMENT PTY LTD,TP-LINK TECHNOLOGIES CO.LTD.,17,143.0,2.0,187.0,1.0,1.0,77.0,1.0,1.0,110.0,77.0,93.5,23.33452377915607,110.0,77.0,77.0,0.0,77.0,110.0,110.0,0.0,110.0,1.3076923076923077,0.013986013986013986,110.0,1.0,77.0,1.0)
2. (LIFI LABS MANAGEMENT PTY LTD,TP-LINK TECHNOLOGIES CO.LTD.,17,75.0,2.0,194.0,1.0,1.0,77.0,1.0,1.0,117.0,77.0,97.0,28.284271247461906,117.0,77.0,77.0,0.0,77.0,117.0,117.0,0.0,117.0,2.5866666666666664,0.02666666666666667,117.0,1.0,77.0,1.0)
3. (LIFI LABS MANAGEMENT PTY LTD,TP-LINK TECHNOLOGIES CO.LTD.,17,78.0,2.0,614.0,1.0,1.0,72.0,1.0,1.0,542.0,72.0,307.0,332.3401871576773,542.0,72.0,72.0,0.0,72.0,542.0,542.0,0.0,542.0,7.871794871794871,0.02564102564102564,542.0,1.0,72.0,1.0)

Examples for computer:
1. (Apple Inc.,Awair,6,2.0,2.0,132.0,1.0,1.0,78.0,1.0,1.0,54.0,54.0,66.0,16.97056274847714,78.0,78.0,78.0,0.0,78.0,54.0,54.0,0.0,54.0,66.0,1.0,54.0,1.0,78.0,1.0)
2. (Apple Inc.,Awair,6,1.0,2.0,132.0,1.0,1.0,78.0,1.0,1.0,54.0,54.0,66.0,16.97056274847714,78.0,78.0,78.0,0.0,78.0,54.0,54.0,0.0,54.0,132.0,2.0,54.0,1.0,78.0,1.0)
3. (Apple Inc.,TP-LINK TECHNOLOGIES CO.LTD.,17,68458.0,6.0,474.0,68458.0,3.0,237.0,68457.0,3.0,237.0,79.0,79.0,0.0,79.0,79.0,79.0,0.0,79.0,79.0,79.0,0.0,79.0,0.006923953372870957,8.764497940342983e-05,0.0034620272579867655,4.382312984793374e-05,0.0034619766864354786,4.382248970171492e-05)

Examples for motion_sensor:
1. (Belkin International Inc.,TP-LINK TECHNOLOGIES CO.LTD.,6,219.0,3.0,426.0,219.0,2.0,280.0,1.0,1.0,146.0,66.0,142.0,74.08103670980854,214.0,66.0,140.0,104.65180361560904,214.0,146.0,146.0,0.0,146.0,1.9452054794520548,0.0136986301369863,146.0,1.0,1.278538812785388,0.0091324200913242)
2. (Belkin International Inc.,TP-LINK TECHNOLOGIES CO.LTD.,6,218.0,3.0,426.0,218.0,2.0,280.0,1.0,1.0,146.0,66.0,142.0,74.08103670980854,214.0,66.0,140.0,104.65180361560904,214.0,146.0,146.0,0.0,146.0,1.9541284403669725,0.013761467889908258,146.0,1.0,1.2844036697247707,0.009174311926605505)
3. (Belkin International Inc.,TP-LINK TECHNOLOGIES CO.LTD.,6,217.0,3.0,426.0,217.0,2.0,280.0,1.0,1.0,146.0,66.0,142.0,74.08103670980854,214.0,66.0,140.0,104.65180361560904,214.0,146.0,146.0,0.0,146.0,1.9631336405529953,0.013824884792626729,146.0,1.0,1.2903225806451613,0.009216589861751152)

Examples for printer:
1. (Hewlett Packard,IPV6 Multicast,58,119918.0,2.0,172.0,119918.0,2.0,172.0,1.0,0.0,0.0,86.0,86.0,0.0,86.0,86.0,86.0,0.0,86.0,0.0,0.0,0.0,0.0,0.0014343134475224737,1.667806334328458e-05,0.0,0.0,0.0014343134475224737,1.667806334328458e-05)
2. (Hewlett Packard,IPV6 Multicast,58,118735.0,2.0,172.0,118735.0,2.0,172.0,1.0,0.0,0.0,86.0,86.0,0.0,86.0,86.0,86.0,0.0,86.0,0.0,0.0,0.0,0.0,0.0014486040341937928,1.6844232955741777e-05,0.0,0.0,0.0014486040341937928,1.6844232955741777e-05)
3. (Hewlett Packard,IPV6 Multicast,58,119651.0,2.0,172.0,119651.0,2.0,172.0,1.0,0.0,0.0,86.0,86.0,0.0,86.0,86.0,86.0,0.0,86.0,0.0,0.0,0.0,0.0,0.0014375141035177308,1.6715280273461986e-05,0.0,0.0,0.0014375141035177308,1.6715280273461986e-05)""" 
}

In [4]:
SYSTEM_DIRECTIVE_PROMPT_TUNING_5_EXAMPLE = { 
    "role": "system", 
    "content": """Your task is to use various attributes of the traffic transmitted/received by different IoT devices to classify them into one of 9 specific device types. You'll be provided a comma-separated tuple with 30 traffic attributes in the following order:

Communication Attributes

1. src_oui: Manufacturer of the device sending the frame.
2. dst_oui: The destination that the frame is headed for.
3. protocol: Protocol number inside the Layer 2 Frame.

Transmission Attributes (Time in milliseconds, Size in bytes, Packets in count)

4. bidirectional_total_duration_ms: Total duration of bidirectional communication between the source and destination.
5. bidirectional_total_packets: Total number of packets transmitted bidirectionally across the source-destination channel.
6. bidirectional_total_bytes: Total bytes transmitted bidirectionally across the source-destination channel.
7. src2dst_total_duration_ms: Total duration of unidirectional communication from source to destination.
8. src2dst_total_packets: Total number of packets transmitted from source to destination.
9. src2dst_total_bytes: Total bytes transmitted from source to destination.
10. dst2src_total_duration_ms: Total duration of unidirectional communication from destination to source.
11. dst2src_total_packets: Total number of packets transmitted from destination to source.
12. dst2src_total_bytes: Total bytes transmitted from destination to source.

Packet Size Attributes (Size in bytes)

13. bidirectional_min_ps: Smallest packet size across bidirectional communication.
14. bidirectional_mean_ps: Mean packet size across bidirectional communication.
15. bidirectional_stdev_ps: Standard deviation in packet size across bidirectional communication.
16. bidirectional_max_ps: Largest packet size across bidirectional communication.
17. src2dst_min_ps: Smallest packet size from source to destination.
18. src2dst_mean_ps: Mean packet size from source to destination.
19. src2dst_stdev_ps: Standard deviation in packet size from source to destination.
20. src2dst_max_ps: Largest packet size from source to destination.
21. dst2src_min_ps: Smallest packet size from destination to source.
22. dst2src_mean_ps: Mean packet size from destination to source.
23. dst2src_stdev_ps: Standard deviation in packet size from destination to source.
24. dst2src_max_ps: Largest packet size from destination to source.

Transmission Rate Attributes (Bytes per millisecond, Packets per millisecond)

25. bidirectional_transmission_rate_byte_ms: Number of bytes transmitted per millisecond bidirectionally.
26. bidirectional_transmission_rate_ms: Number of packets transmitted per millisecond bidirectionally.
27. dst2src_transmission_rate_bytes_ms: Number of bytes transmitted per millisecond from destination to source.
28. dst2src_transmission_rate_ms: Number of packets transmitted per millisecond from destination to source.
29. src2dst_transmission_rate_bytes_ms: Number of bytes transmitted per millisecond from source to destination.
30. src2dst_transmission_rate_ms: Number of packets transmitted per millisecond from source to destination.

You can only classify the traffic into only one of the following 9 IoT device types, exactly as they appear below:

camera
router
mobile
switch
speaker
bulb
computer
motion_sensor
printer

Note: Not all attributes might be available for every frame, and some may be more telling than others in identifying the device type.

Now, you will be given 5 examples of tuples for each type of device. The order of the attributes in the tuples will be same as the order in which they were defined above.

Examples for camera:
1. (Insteon,Multicast,2,112030.0,4.0,240.0,112030.0,4.0,240.0,1.0,0.0,0.0,60.0,60.0,0.0,60.0,60.0,60.0,0.0,60.0,0.0,0.0,0.0,0.0,0.0021422833169686692,3.5704721949477816e-05,0.0,0.0,0.0021422833169686692,3.5704721949477816e-05)
2. (Insteon,TP-LINK TECHNOLOGIES CO.LTD.,17,216.0,2.0,340.0,1.0,1.0,78.0,1.0,1.0,262.0,78.0,170.0,130.10764773832474,262.0,78.0,78.0,0.0,78.0,262.0,262.0,0.0,262.0,1.5740740740740742,0.009259259259259259,262.0,1.0,78.0,1.0)
3. (Samsung Electronics Co.Ltd,Multicast,2,1.0,1.0,46.0,1.0,1.0,46.0,1.0,0.0,0.0,46.0,46.0,0.0,46.0,46.0,46.0,0.0,46.0,0.0,0.0,0.0,0.0,46.0,1.0,0.0,0.0,46.0,1.0)
4. (Samsung Electronics Co.Ltd,Belkin International Inc.,17,204.0,2.0,762.0,204.0,2.0,762.0,1.0,0.0,0.0,381.0,381.0,0.0,381.0,381.0,381.0,0.0,381.0,0.0,0.0,0.0,0.0,3.735294117647059,0.00980392156862745,0.0,0.0,3.735294117647059,0.00980392156862745)
5. (Netatmo,TP-LINK TECHNOLOGIES CO.LTD.,1,1313.0,4.0,392.0,1003.0,2.0,196.0,1003.0,2.0,196.0,98.0,98.0,0.0,98.0,98.0,98.0,0.0,98.0,98.0,98.0,0.0,98.0,0.2985529322162985,0.0030464584920030465,0.19541375872382852,0.0019940179461615153,0.19541375872382852,0.0019940179461615153)

Examples for router:
1. (TP-LINK TECHNOLOGIES CO.LTD.,Invoxia,6,1740517.0,102.0,15258.0,1740477.0,42.0,6924.0,1740384.0,60.0,8334.0,66.0,149.5882352941177,227.36559964037056,785.0,66.0,164.85714285714292,243.02058242939128,753.0,66.0,138.90000000000003,217.19090251572047,785.0,0.008766360799693425,5.8603277072272204e-05,0.0047885983782889295,3.447515031165536e-05,0.003978219763892312,2.4131315725516626e-05)
2. (TP-LINK TECHNOLOGIES CO.LTD.,Samsung Electronics Co.Ltd,6,696.0,4.0,318.0,696.0,2.0,155.0,8.0,2.0,163.0,66.0,79.5,15.92691642053372,97.0,66.0,77.5,16.263455967290593,89.0,66.0,81.5,21.920310216782973,97.0,0.45689655172413796,0.005747126436781609,20.375,0.25,0.22270114942528735,0.0028735632183908046)
3. (TP-LINK TECHNOLOGIES CO.LTD.,Netatmo,17,8117.0,4.0,1376.0,81.0,3.0,1026.0,1.0,1.0,350.0,342.0,344.0,4.0,350.0,342.0,342.0,0.0,342.0,350.0,350.0,0.0,350.0,0.16952075890107182,0.0004927929037821855,350.0,1.0,12.666666666666666,0.037037037037037035)
4. (TP-LINK TECHNOLOGIES CO.LTD.,Netatmo,17,83026.0,14.0,4796.0,74753.0,13.0,4446.0,1.0,1.0,350.0,342.0,342.57142857142856,2.1380899352993974,350.0,342.0,342.0,0.0,342.0,350.0,350.0,0.0,350.0,0.05776503745814564,0.0001686218774841616,350.0,1.0,0.05947587387797145,0.00017390606397067675)
5. (TP-LINK TECHNOLOGIES CO.LTD.,IPV6 Multicast,58,119040.0,2.0,172.0,119040.0,2.0,172.0,1.0,0.0,0.0,86.0,86.0,0.0,86.0,86.0,86.0,0.0,86.0,0.0,0.0,0.0,0.0,0.0014448924731182795,1.6801075268817204e-05,0.0,0.0,0.0014448924731182795,1.6801075268817204e-05)

Examples for mobile:
1. (Samsung Electronics Co.Ltd,TP-LINK TECHNOLOGIES CO.LTD.,6,1.0,1.0,97.0,1.0,1.0,97.0,1.0,0.0,0.0,97.0,97.0,0.0,97.0,97.0,97.0,0.0,97.0,0.0,0.0,0.0,0.0,97.0,1.0,0.0,0.0,97.0,1.0)
2. (Samsung Electronics Co.Ltd,TP-LINK TECHNOLOGIES CO.LTD.,58,5770.0,4.0,336.0,5770.0,2.0,172.0,5009.0,2.0,164.0,78.0,84.0,4.000000000000001,86.0,86.0,86.0,0.0,86.0,78.0,82.0,5.656854249492381,86.0,0.058232235701906415,0.0006932409012131716,0.0327410660810541,0.0003992812936713915,0.02980935875216638,0.0003466204506065858)
3. (Samsung Electronics Co.Ltd,TP-LINK TECHNOLOGIES CO.LTD.,6,143.0,4.0,314.0,143.0,2.0,157.0,1.0,2.0,157.0,66.0,78.5,14.433756729740644,91.0,66.0,78.5,17.67766952966369,91.0,66.0,78.5,17.67766952966369,91.0,2.195804195804196,0.027972027972027972,157.0,2.0,1.097902097902098,0.013986013986013986)
4. (AMPAK Technology Inc.,IPV6 Multicast,58,1.0,1.0,86.0,1.0,1.0,86.0,1.0,0.0,0.0,86.0,86.0,0.0,86.0,86.0,86.0,0.0,86.0,0.0,0.0,0.0,0.0,86.0,1.0,0.0,0.0,86.0,1.0)
5. (HTC Corporation,TP-LINK TECHNOLOGIES CO.LTD.,6,172.0,10.0,1202.0,172.0,6.0,764.0,169.0,4.0,438.0,66.0,120.2,74.0342263190929,246.0,66.0,127.33333333333331,91.97100992522952,246.0,66.0,109.5,45.727453460694704,149.0,6.988372093023256,0.05813953488372093,2.591715976331361,0.023668639053254437,4.441860465116279,0.03488372093023256)

Examples for switch:
1. (Belkin International Inc.,TP-LINK TECHNOLOGIES CO.LTD.,6,219.0,3.0,426.0,219.0,2.0,280.0,1.0,1.0,146.0,66.0,142.0,74.08103670980854,214.0,66.0,140.0,104.65180361560904,214.0,146.0,146.0,0.0,146.0,1.9452054794520548,0.0136986301369863,146.0,1.0,1.278538812785388,0.0091324200913242)
2. (Belkin International Inc.,TP-LINK TECHNOLOGIES CO.LTD.,6,217.0,3.0,426.0,217.0,2.0,280.0,1.0,1.0,146.0,66.0,142.0,74.08103670980854,214.0,66.0,140.0,104.65180361560904,214.0,146.0,146.0,0.0,146.0,1.9631336405529953,0.013824884792626729,146.0,1.0,1.2903225806451613,0.009216589861751152)
3. (Belkin International Inc.,Belkin International Inc.,6,226.0,13.0,1386.0,226.0,7.0,789.0,219.0,6.0,597.0,66.0,106.61538461538464,75.45698384017486,259.0,66.0,112.71428571428572,78.7204215048723,253.0,66.0,99.5,78.20421983499355,259.0,6.132743362831858,0.05752212389380531,2.7260273972602738,0.0273972602739726,3.4911504424778763,0.030973451327433628)
4. (Belkin International Inc.,IPV6 Multicast,58,118736.0,2.0,172.0,118736.0,2.0,172.0,1.0,0.0,0.0,86.0,86.0,0.0,86.0,86.0,86.0,0.0,86.0,0.0,0.0,0.0,0.0,0.0014485918339846382,1.684409109284463e-05,0.0,0.0,0.0014485918339846382,1.684409109284463e-05)
5. (Belkin International Inc.,Multicast,2,118200.0,2.0,92.0,118200.0,2.0,92.0,1.0,0.0,0.0,46.0,46.0,0.0,46.0,46.0,46.0,0.0,46.0,0.0,0.0,0.0,0.0,0.00077834179357022,1.6920473773265652e-05,0.0,0.0,0.00077834179357022,1.6920473773265652e-05)

Examples for speaker:
1. (Amazon Technologies Inc.,TP-LINK TECHNOLOGIES CO.LTD.,6,1995.0,20.0,7599.0,1995.0,11.0,1469.0,1742.0,9.0,6130.0,66.0,379.9500000000001,507.56420128004754,1484.0,66.0,133.54545454545456,155.40486712864796,583.0,66.0,681.1111111111111,629.3996036788641,1484.0,3.8090225563909774,0.010025062656641603,3.51894374282434,0.005166475315729047,0.7363408521303259,0.005513784461152882)
2. (Amazon Technologies Inc.,TP-LINK TECHNOLOGIES CO.LTD.,6,147372.0,28.0,2187.0,147372.0,15.0,1046.0,147163.0,13.0,1141.0,54.0,78.10714285714286,40.216395482545,257.0,54.0,69.73333333333333,20.09430149024724,95.0,54.0,87.76923076923076,54.58197786533855,257.0,0.014839996742936243,0.00018999538582634422,0.007753307556926673,8.833742177041783e-05,0.007097684770512716,0.00010178324240697012)
3. (Amazon Technologies Inc.,TP-LINK TECHNOLOGIES CO.LTD.,17,191.0,2.0,180.0,1.0,1.0,90.0,1.0,1.0,90.0,90.0,90.0,0.0,90.0,90.0,90.0,0.0,90.0,90.0,90.0,0.0,90.0,0.9424083769633508,0.010471204188481676,90.0,1.0,90.0,1.0)
4. (Amazon Technologies Inc.,TP-LINK TECHNOLOGIES CO.LTD.,17,309224.0,14.0,1260.0,309007.0,7.0,630.0,309007.0,7.0,630.0,90.0,90.0,0.0,90.0,90.0,90.0,0.0,90.0,90.0,90.0,0.0,90.0,0.0040747160634362145,4.527462292706905e-05,0.0020387887653030515,2.265320850336724e-05,0.0020387887653030515,2.265320850336724e-05)
5. (Invoxia,TP-LINK TECHNOLOGIES CO.LTD.,6,303.0,2.0,132.0,1.0,1.0,66.0,1.0,1.0,66.0,66.0,66.0,0.0,66.0,66.0,66.0,0.0,66.0,66.0,66.0,0.0,66.0,0.43564356435643564,0.006600660066006601,66.0,1.0,66.0,1.0)

Examples for bulb:
1. (LIFI LABS MANAGEMENT PTY LTD,TP-LINK TECHNOLOGIES CO.LTD.,17,143.0,2.0,187.0,1.0,1.0,77.0,1.0,1.0,110.0,77.0,93.5,23.33452377915607,110.0,77.0,77.0,0.0,77.0,110.0,110.0,0.0,110.0,1.3076923076923077,0.013986013986013986,110.0,1.0,77.0,1.0)
2. (LIFI LABS MANAGEMENT PTY LTD,TP-LINK TECHNOLOGIES CO.LTD.,17,75.0,2.0,194.0,1.0,1.0,77.0,1.0,1.0,117.0,77.0,97.0,28.284271247461906,117.0,77.0,77.0,0.0,77.0,117.0,117.0,0.0,117.0,2.5866666666666664,0.02666666666666667,117.0,1.0,77.0,1.0)
3. (LIFI LABS MANAGEMENT PTY LTD,TP-LINK TECHNOLOGIES CO.LTD.,17,78.0,2.0,614.0,1.0,1.0,72.0,1.0,1.0,542.0,72.0,307.0,332.3401871576773,542.0,72.0,72.0,0.0,72.0,542.0,542.0,0.0,542.0,7.871794871794871,0.02564102564102564,542.0,1.0,72.0,1.0)
4. (LIFI LABS MANAGEMENT PTY LTD,TP-LINK TECHNOLOGIES CO.LTD.,17,247.0,2.0,180.0,1.0,1.0,90.0,1.0,1.0,90.0,90.0,90.0,0.0,90.0,90.0,90.0,0.0,90.0,90.0,90.0,0.0,90.0,0.728744939271255,0.008097165991902834,90.0,1.0,90.0,1.0)
5. (LIFI LABS MANAGEMENT PTY LTD,TP-LINK TECHNOLOGIES CO.LTD.,6,120574.0,148.0,17022.0,120401.0,66.0,9933.0,120401.0,82.0,7089.0,54.0,115.01351351351352,82.3665266604211,784.0,54.0,150.49999999999997,47.48351940815483,321.0,54.0,86.45121951219512,93.01004226134614,784.0,0.14117471428334466,0.0012274619735598057,0.058878248519530565,0.0006810574663001138,0.08249931478974427,0.0005481682045830184)

Examples for computer:
1. (Apple Inc.,Awair,6,2.0,2.0,132.0,1.0,1.0,78.0,1.0,1.0,54.0,54.0,66.0,16.97056274847714,78.0,78.0,78.0,0.0,78.0,54.0,54.0,0.0,54.0,66.0,1.0,54.0,1.0,78.0,1.0)
2. (Apple Inc.,Awair,6,1.0,2.0,132.0,1.0,1.0,78.0,1.0,1.0,54.0,54.0,66.0,16.97056274847714,78.0,78.0,78.0,0.0,78.0,54.0,54.0,0.0,54.0,132.0,2.0,54.0,1.0,78.0,1.0)
3. (Apple Inc.,TP-LINK TECHNOLOGIES CO.LTD.,17,68458.0,6.0,474.0,68458.0,3.0,237.0,68457.0,3.0,237.0,79.0,79.0,0.0,79.0,79.0,79.0,0.0,79.0,79.0,79.0,0.0,79.0,0.006923953372870957,8.764497940342983e-05,0.0034620272579867655,4.382312984793374e-05,0.0034619766864354786,4.382248970171492e-05)
4. (AzureWave Technology Inc.,TP-LINK TECHNOLOGIES CO.LTD.,6,804992.0,62.0,32305.0,804992.0,24.0,21889.0,804821.0,38.0,10416.0,54.0,521.0483870967743,582.9122034788508,1514.0,54.0,912.0416666666669,650.1257723914167,1404.0,54.0,274.1052631578947,368.3209374234968,1514.0,0.04013083359834632,7.701939895054857e-05,0.012942008222946468,4.7215467787247104e-05,0.027191574574654157,2.981396088408332e-05)
5. (AzureWave Technology Inc.,Broadcast,17,86223.0,147.0,13956.0,86223.0,147.0,13956.0,1.0,0.0,0.0,92.0,94.9387755102041,6.6756811666151625,110.0,92.0,94.9387755102041,6.6756811666151625,110.0,0.0,0.0,0.0,0.0,0.16185936467067952,0.0017048815281305453,0.0,0.0,0.16185936467067952,0.0017048815281305453)

Examples for motion_sensor:
1. (Belkin International Inc.,TP-LINK TECHNOLOGIES CO.LTD.,6,219.0,3.0,426.0,219.0,2.0,280.0,1.0,1.0,146.0,66.0,142.0,74.08103670980854,214.0,66.0,140.0,104.65180361560904,214.0,146.0,146.0,0.0,146.0,1.9452054794520548,0.0136986301369863,146.0,1.0,1.278538812785388,0.0091324200913242)
2. (Belkin International Inc.,TP-LINK TECHNOLOGIES CO.LTD.,6,218.0,3.0,426.0,218.0,2.0,280.0,1.0,1.0,146.0,66.0,142.0,74.08103670980854,214.0,66.0,140.0,104.65180361560904,214.0,146.0,146.0,0.0,146.0,1.9541284403669725,0.013761467889908258,146.0,1.0,1.2844036697247707,0.009174311926605505)
3. (Belkin International Inc.,TP-LINK TECHNOLOGIES CO.LTD.,6,217.0,3.0,426.0,217.0,2.0,280.0,1.0,1.0,146.0,66.0,142.0,74.08103670980854,214.0,66.0,140.0,104.65180361560904,214.0,146.0,146.0,0.0,146.0,1.9631336405529953,0.013824884792626729,146.0,1.0,1.2903225806451613,0.009216589861751152)
4. (Belkin International Inc.,Samsung Electronics Co.Ltd,6,57.0,10.0,2032.0,56.0,5.0,524.0,53.0,5.0,1508.0,66.0,203.2,367.4817606962888,1236.0,66.0,104.8,82.36018455540274,252.0,66.0,301.6,522.3569660682242,1236.0,35.64912280701754,0.17543859649122806,28.452830188679247,0.09433962264150944,9.357142857142858,0.08928571428571429)
5. (Belkin International Inc.,IPV6 Multicast,58,119868.0,2.0,172.0,119868.0,2.0,172.0,1.0,0.0,0.0,86.0,86.0,0.0,86.0,86.0,86.0,0.0,86.0,0.0,0.0,0.0,0.0,0.0014349117362432008,1.6685020188874427e-05,0.0,0.0,0.0014349117362432008,1.6685020188874427e-05)

Examples for printer:
1. (Hewlett Packard,IPV6 Multicast,58,119918.0,2.0,172.0,119918.0,2.0,172.0,1.0,0.0,0.0,86.0,86.0,0.0,86.0,86.0,86.0,0.0,86.0,0.0,0.0,0.0,0.0,0.0014343134475224737,1.667806334328458e-05,0.0,0.0,0.0014343134475224737,1.667806334328458e-05)
2. (Hewlett Packard,IPV6 Multicast,58,118735.0,2.0,172.0,118735.0,2.0,172.0,1.0,0.0,0.0,86.0,86.0,0.0,86.0,86.0,86.0,0.0,86.0,0.0,0.0,0.0,0.0,0.0014486040341937928,1.6844232955741777e-05,0.0,0.0,0.0014486040341937928,1.6844232955741777e-05)
3. (Hewlett Packard,IPV6 Multicast,58,119651.0,2.0,172.0,119651.0,2.0,172.0,1.0,0.0,0.0,86.0,86.0,0.0,86.0,86.0,86.0,0.0,86.0,0.0,0.0,0.0,0.0,0.0014375141035177308,1.6715280273461986e-05,0.0,0.0,0.0014375141035177308,1.6715280273461986e-05)
4. (Hewlett Packard,Multicast,2,116386.0,2.0,92.0,116386.0,2.0,92.0,1.0,0.0,0.0,46.0,46.0,0.0,46.0,46.0,46.0,0.0,46.0,0.0,0.0,0.0,0.0,0.0007904730809547541,1.718419741205987e-05,0.0,0.0,0.0007904730809547541,1.718419741205987e-05)
5. (Hewlett Packard,TP-LINK TECHNOLOGIES CO.LTD.,6,1791976.0,50.0,5198.0,1791777.0,25.0,3500.0,1791776.0,25.0,1698.0,66.0,103.96,36.541259877970646,140.0,140.0,140.0,0.0,140.0,66.0,67.92,4.489988864128729,78.0,0.0029007084916315845,2.790215940392059e-05,0.000947663100744736,1.3952636936759951e-05,0.001953368080960968,1.3952629149721198e-05)""" 
}

In [5]:
SYSTEM_DIRECTIVE_PROMPT_TUNING_7_EXAMPLE = { 
    "role": "system", 
    "content": """Your task is to use various attributes of the traffic transmitted/received by different IoT devices to classify them into one of 9 specific device types. You'll be provided a comma-separated tuple with 30 traffic attributes in the following order:

Communication Attributes

1. src_oui: Manufacturer of the device sending the frame.
2. dst_oui: The destination that the frame is headed for.
3. protocol: Protocol number inside the Layer 2 Frame.

Transmission Attributes (Time in milliseconds, Size in bytes, Packets in count)

4. bidirectional_total_duration_ms: Total duration of bidirectional communication between the source and destination.
5. bidirectional_total_packets: Total number of packets transmitted bidirectionally across the source-destination channel.
6. bidirectional_total_bytes: Total bytes transmitted bidirectionally across the source-destination channel.
7. src2dst_total_duration_ms: Total duration of unidirectional communication from source to destination.
8. src2dst_total_packets: Total number of packets transmitted from source to destination.
9. src2dst_total_bytes: Total bytes transmitted from source to destination.
10. dst2src_total_duration_ms: Total duration of unidirectional communication from destination to source.
11. dst2src_total_packets: Total number of packets transmitted from destination to source.
12. dst2src_total_bytes: Total bytes transmitted from destination to source.

Packet Size Attributes (Size in bytes)

13. bidirectional_min_ps: Smallest packet size across bidirectional communication.
14. bidirectional_mean_ps: Mean packet size across bidirectional communication.
15. bidirectional_stdev_ps: Standard deviation in packet size across bidirectional communication.
16. bidirectional_max_ps: Largest packet size across bidirectional communication.
17. src2dst_min_ps: Smallest packet size from source to destination.
18. src2dst_mean_ps: Mean packet size from source to destination.
19. src2dst_stdev_ps: Standard deviation in packet size from source to destination.
20. src2dst_max_ps: Largest packet size from source to destination.
21. dst2src_min_ps: Smallest packet size from destination to source.
22. dst2src_mean_ps: Mean packet size from destination to source.
23. dst2src_stdev_ps: Standard deviation in packet size from destination to source.
24. dst2src_max_ps: Largest packet size from destination to source.

Transmission Rate Attributes (Bytes per millisecond, Packets per millisecond)

25. bidirectional_transmission_rate_byte_ms: Number of bytes transmitted per millisecond bidirectionally.
26. bidirectional_transmission_rate_ms: Number of packets transmitted per millisecond bidirectionally.
27. dst2src_transmission_rate_bytes_ms: Number of bytes transmitted per millisecond from destination to source.
28. dst2src_transmission_rate_ms: Number of packets transmitted per millisecond from destination to source.
29. src2dst_transmission_rate_bytes_ms: Number of bytes transmitted per millisecond from source to destination.
30. src2dst_transmission_rate_ms: Number of packets transmitted per millisecond from source to destination.

You can only classify the traffic into only one of the following 9 IoT device types, exactly as they appear below:

camera
router
mobile
switch
speaker
bulb
computer
motion_sensor
printer

Note: Not all attributes might be available for every frame, and some may be more telling than others in identifying the device type.

Now, you will be given 5 examples of tuples for each type of device. The order of the attributes in the tuples will be same as the order in which they were defined above.

Examples for camera:
1. (Insteon,Multicast,2,112030.0,4.0,240.0,112030.0,4.0,240.0,1.0,0.0,0.0,60.0,60.0,0.0,60.0,60.0,60.0,0.0,60.0,0.0,0.0,0.0,0.0,0.0021422833169686692,3.5704721949477816e-05,0.0,0.0,0.0021422833169686692,3.5704721949477816e-05)
2. (Insteon,TP-LINK TECHNOLOGIES CO.LTD.,17,216.0,2.0,340.0,1.0,1.0,78.0,1.0,1.0,262.0,78.0,170.0,130.10764773832474,262.0,78.0,78.0,0.0,78.0,262.0,262.0,0.0,262.0,1.5740740740740742,0.009259259259259259,262.0,1.0,78.0,1.0)
3. (Samsung Electronics Co.Ltd,Multicast,2,1.0,1.0,46.0,1.0,1.0,46.0,1.0,0.0,0.0,46.0,46.0,0.0,46.0,46.0,46.0,0.0,46.0,0.0,0.0,0.0,0.0,46.0,1.0,0.0,0.0,46.0,1.0)
4. (Samsung Electronics Co.Ltd,Belkin International Inc.,17,204.0,2.0,762.0,204.0,2.0,762.0,1.0,0.0,0.0,381.0,381.0,0.0,381.0,381.0,381.0,0.0,381.0,0.0,0.0,0.0,0.0,3.735294117647059,0.00980392156862745,0.0,0.0,3.735294117647059,0.00980392156862745)
5. (Netatmo,TP-LINK TECHNOLOGIES CO.LTD.,1,1313.0,4.0,392.0,1003.0,2.0,196.0,1003.0,2.0,196.0,98.0,98.0,0.0,98.0,98.0,98.0,0.0,98.0,98.0,98.0,0.0,98.0,0.2985529322162985,0.0030464584920030465,0.19541375872382852,0.0019940179461615153,0.19541375872382852,0.0019940179461615153)
6. (Insteon,TP-LINK TECHNOLOGIES CO.LTD.,17,1.0,1.0,274.0,1.0,1.0,274.0,1.0,0.0,0.0,274.0,274.0,0.0,274.0,274.0,274.0,0.0,274.0,0.0,0.0,0.0,0.0,274.0,1.0,0.0,0.0,274.0,1.0)
7. (Netatmo,IPV6 Multicast,58,118439.0,2.0,172.0,118439.0,2.0,172.0,1.0,0.0,0.0,86.0,86.0,0.0,86.0,86.0,86.0,0.0,86.0,0.0,0.0,0.0,0.0,0.001452224351776,1.6886329671814184e-05,0.0,0.0,0.001452224351776,1.6886329671814184e-05)

Examples for router:
1. (TP-LINK TECHNOLOGIES CO.LTD.,Invoxia,6,1740517.0,102.0,15258.0,1740477.0,42.0,6924.0,1740384.0,60.0,8334.0,66.0,149.5882352941177,227.36559964037056,785.0,66.0,164.85714285714292,243.02058242939128,753.0,66.0,138.90000000000003,217.19090251572047,785.0,0.008766360799693425,5.8603277072272204e-05,0.0047885983782889295,3.447515031165536e-05,0.003978219763892312,2.4131315725516626e-05)
2. (TP-LINK TECHNOLOGIES CO.LTD.,Samsung Electronics Co.Ltd,6,696.0,4.0,318.0,696.0,2.0,155.0,8.0,2.0,163.0,66.0,79.5,15.92691642053372,97.0,66.0,77.5,16.263455967290593,89.0,66.0,81.5,21.920310216782973,97.0,0.45689655172413796,0.005747126436781609,20.375,0.25,0.22270114942528735,0.0028735632183908046)
3. (TP-LINK TECHNOLOGIES CO.LTD.,Netatmo,17,8117.0,4.0,1376.0,81.0,3.0,1026.0,1.0,1.0,350.0,342.0,344.0,4.0,350.0,342.0,342.0,0.0,342.0,350.0,350.0,0.0,350.0,0.16952075890107182,0.0004927929037821855,350.0,1.0,12.666666666666666,0.037037037037037035)
4. (TP-LINK TECHNOLOGIES CO.LTD.,Netatmo,17,83026.0,14.0,4796.0,74753.0,13.0,4446.0,1.0,1.0,350.0,342.0,342.57142857142856,2.1380899352993974,350.0,342.0,342.0,0.0,342.0,350.0,350.0,0.0,350.0,0.05776503745814564,0.0001686218774841616,350.0,1.0,0.05947587387797145,0.00017390606397067675)
5. (TP-LINK TECHNOLOGIES CO.LTD.,IPV6 Multicast,58,119040.0,2.0,172.0,119040.0,2.0,172.0,1.0,0.0,0.0,86.0,86.0,0.0,86.0,86.0,86.0,0.0,86.0,0.0,0.0,0.0,0.0,0.0014448924731182795,1.6801075268817204e-05,0.0,0.0,0.0014448924731182795,1.6801075268817204e-05)
6. (TP-LINK TECHNOLOGIES CO.LTD.,Amazon Technologies Inc.,6,51.0,2.0,132.0,1.0,1.0,66.0,1.0,1.0,66.0,66.0,66.0,0.0,66.0,66.0,66.0,0.0,66.0,66.0,66.0,0.0,66.0,2.588235294117647,0.0392156862745098,66.0,1.0,66.0,1.0)
7. (TP-LINK TECHNOLOGIES CO.LTD.,Netatmo,17,1797460.0,200.0,13770.0,1797460.0,100.0,6957.0,1797101.0,100.0,6813.0,43.0,68.84999999999995,56.19248595129899,394.0,43.0,69.57,59.50829752855878,394.0,43.0,68.12999999999997,52.960373026547984,250.0,0.0076608102544701,0.0001112681227955,0.0037911057864861,5.564517520161638e-05,0.0038704616514414,5.563406139775016e-05)

Examples for mobile:
1. (Samsung Electronics Co.Ltd,TP-LINK TECHNOLOGIES CO.LTD.,6,1.0,1.0,97.0,1.0,1.0,97.0,1.0,0.0,0.0,97.0,97.0,0.0,97.0,97.0,97.0,0.0,97.0,0.0,0.0,0.0,0.0,97.0,1.0,0.0,0.0,97.0,1.0)
2. (Samsung Electronics Co.Ltd,TP-LINK TECHNOLOGIES CO.LTD.,58,5770.0,4.0,336.0,5770.0,2.0,172.0,5009.0,2.0,164.0,78.0,84.0,4.000000000000001,86.0,86.0,86.0,0.0,86.0,78.0,82.0,5.656854249492381,86.0,0.058232235701906415,0.0006932409012131716,0.0327410660810541,0.0003992812936713915,0.02980935875216638,0.0003466204506065858)
3. (Samsung Electronics Co.Ltd,TP-LINK TECHNOLOGIES CO.LTD.,6,143.0,4.0,314.0,143.0,2.0,157.0,1.0,2.0,157.0,66.0,78.5,14.433756729740644,91.0,66.0,78.5,17.67766952966369,91.0,66.0,78.5,17.67766952966369,91.0,2.195804195804196,0.027972027972027972,157.0,2.0,1.097902097902098,0.013986013986013986)
4. (AMPAK Technology Inc.,IPV6 Multicast,58,1.0,1.0,86.0,1.0,1.0,86.0,1.0,0.0,0.0,86.0,86.0,0.0,86.0,86.0,86.0,0.0,86.0,0.0,0.0,0.0,0.0,86.0,1.0,0.0,0.0,86.0,1.0)
5. (HTC Corporation,TP-LINK TECHNOLOGIES CO.LTD.,6,172.0,10.0,1202.0,172.0,6.0,764.0,169.0,4.0,438.0,66.0,120.2,74.0342263190929,246.0,66.0,127.33333333333331,91.97100992522952,246.0,66.0,109.5,45.727453460694704,149.0,6.988372093023256,0.05813953488372093,2.591715976331361,0.023668639053254437,4.441860465116279,0.03488372093023256)
6. (Samsung Electronics Co.Ltd,TP-LINK TECHNOLOGIES CO.LTD.,6,568.0,17.0,3524.0,568.0,8.0,1621.0,392.0,9.0,1903.0,66.0,207.29411764705884,230.13658029143323,894.0,66.0,202.625,287.6291945742444,894.0,66.0,211.44444444444449,183.01715159453715,547.0,6.204225352112676,0.0299295774647887,4.854591836734694,0.0229591836734693,2.8538732394366195,0.0140845070422535)
7. (Samsung Electronics Co.Ltd,TP-LINK TECHNOLOGIES CO.LTD.,6,2496.0,24.0,5721.0,2496.0,12.0,3061.0,2494.0,12.0,2660.0,66.0,238.375,301.0856894495392,1263.0,66.0,255.0833333333333,395.1129777057635,1263.0,66.0,221.66666666666663,181.17110613166133,519.0,2.2920673076923075,0.0096153846153846,1.0665597433841218,0.0048115477145148,1.2263621794871795,0.0048076923076923)

Examples for switch:
1. (Belkin International Inc.,TP-LINK TECHNOLOGIES CO.LTD.,6,219.0,3.0,426.0,219.0,2.0,280.0,1.0,1.0,146.0,66.0,142.0,74.08103670980854,214.0,66.0,140.0,104.65180361560904,214.0,146.0,146.0,0.0,146.0,1.9452054794520548,0.0136986301369863,146.0,1.0,1.278538812785388,0.0091324200913242)
2. (Belkin International Inc.,TP-LINK TECHNOLOGIES CO.LTD.,6,217.0,3.0,426.0,217.0,2.0,280.0,1.0,1.0,146.0,66.0,142.0,74.08103670980854,214.0,66.0,140.0,104.65180361560904,214.0,146.0,146.0,0.0,146.0,1.9631336405529953,0.013824884792626729,146.0,1.0,1.2903225806451613,0.009216589861751152)
3. (Belkin International Inc.,Belkin International Inc.,6,226.0,13.0,1386.0,226.0,7.0,789.0,219.0,6.0,597.0,66.0,106.61538461538464,75.45698384017486,259.0,66.0,112.71428571428572,78.7204215048723,253.0,66.0,99.5,78.20421983499355,259.0,6.132743362831858,0.05752212389380531,2.7260273972602738,0.0273972602739726,3.4911504424778763,0.030973451327433628)
4. (Belkin International Inc.,IPV6 Multicast,58,118736.0,2.0,172.0,118736.0,2.0,172.0,1.0,0.0,0.0,86.0,86.0,0.0,86.0,86.0,86.0,0.0,86.0,0.0,0.0,0.0,0.0,0.0014485918339846382,1.684409109284463e-05,0.0,0.0,0.0014485918339846382,1.684409109284463e-05)
5. (Belkin International Inc.,Multicast,2,118200.0,2.0,92.0,118200.0,2.0,92.0,1.0,0.0,0.0,46.0,46.0,0.0,46.0,46.0,46.0,0.0,46.0,0.0,0.0,0.0,0.0,0.00077834179357022,1.6920473773265652e-05,0.0,0.0,0.00077834179357022,1.6920473773265652e-05)
6. (Belkin International Inc.,Belkin International Inc.,6,226.0,13.0,1394.0,226.0,7.0,797.0,219.0,6.0,597.0,66.0,107.23076923076924,76.29236948100144,259.0,66.0,113.85714285714285,80.20895330620012,253.0,66.0,99.5,78.20421983499355,259.0,6.168141592920354,0.0575221238938053,2.7260273972602738,0.0273972602739726,3.5265486725663715,0.0309734513274336)
7. (Belkin International Inc.,IPV6 Multicast,58,119918.0,2.0,172.0,119918.0,2.0,172.0,1.0,0.0,0.0,86.0,86.0,0.0,86.0,86.0,86.0,0.0,86.0,0.0,0.0,0.0,0.0,0.0014343134475224,1.667806334328458e-05,0.0,0.0,0.0014343134475224,1.667806334328458e-05)

Examples for speaker:
1. (Amazon Technologies Inc.,TP-LINK TECHNOLOGIES CO.LTD.,6,1995.0,20.0,7599.0,1995.0,11.0,1469.0,1742.0,9.0,6130.0,66.0,379.9500000000001,507.56420128004754,1484.0,66.0,133.54545454545456,155.40486712864796,583.0,66.0,681.1111111111111,629.3996036788641,1484.0,3.8090225563909774,0.010025062656641603,3.51894374282434,0.005166475315729047,0.7363408521303259,0.005513784461152882)
2. (Amazon Technologies Inc.,TP-LINK TECHNOLOGIES CO.LTD.,6,147372.0,28.0,2187.0,147372.0,15.0,1046.0,147163.0,13.0,1141.0,54.0,78.10714285714286,40.216395482545,257.0,54.0,69.73333333333333,20.09430149024724,95.0,54.0,87.76923076923076,54.58197786533855,257.0,0.014839996742936243,0.00018999538582634422,0.007753307556926673,8.833742177041783e-05,0.007097684770512716,0.00010178324240697012)
3. (Amazon Technologies Inc.,TP-LINK TECHNOLOGIES CO.LTD.,17,191.0,2.0,180.0,1.0,1.0,90.0,1.0,1.0,90.0,90.0,90.0,0.0,90.0,90.0,90.0,0.0,90.0,90.0,90.0,0.0,90.0,0.9424083769633508,0.010471204188481676,90.0,1.0,90.0,1.0)
4. (Amazon Technologies Inc.,TP-LINK TECHNOLOGIES CO.LTD.,17,309224.0,14.0,1260.0,309007.0,7.0,630.0,309007.0,7.0,630.0,90.0,90.0,0.0,90.0,90.0,90.0,0.0,90.0,90.0,90.0,0.0,90.0,0.0040747160634362145,4.527462292706905e-05,0.0020387887653030515,2.265320850336724e-05,0.0020387887653030515,2.265320850336724e-05)
5. (Invoxia,TP-LINK TECHNOLOGIES CO.LTD.,6,303.0,2.0,132.0,1.0,1.0,66.0,1.0,1.0,66.0,66.0,66.0,0.0,66.0,66.0,66.0,0.0,66.0,66.0,66.0,0.0,66.0,0.43564356435643564,0.006600660066006601,66.0,1.0,66.0,1.0)
6. (Amazon Technologies Inc.,TP-LINK TECHNOLOGIES CO.LTD.,6,48702.0,26.0,7627.0,2730.0,14.0,3735.0,48486.0,12.0,3892.0,54.0,293.34615384615387,470.2936055110843,1514.0,54.0,266.7857142857143,400.6951994435956,1514.0,54.0,324.33333333333337,557.6547210225947,1514.0,0.156605478214447,0.0005338589790973,0.0802705935734026,0.0002474941220146,1.3681318681318682,0.0051282051282051)
7. (Amazon Technologies Inc.,TP-LINK TECHNOLOGIES CO.LTD.,1,57500.0,20.0,1960.0,57500.0,10.0,980.0,57500.0,10.0,980.0,98.0,98.0,0.0,98.0,98.0,98.0,0.0,98.0,98.0,98.0,0.0,98.0,0.0340869565217391,0.0003478260869565,0.0170434782608695,0.0001739130434782,0.0170434782608695,0.0001739130434782)

Examples for bulb:
1. (LIFI LABS MANAGEMENT PTY LTD,TP-LINK TECHNOLOGIES CO.LTD.,17,143.0,2.0,187.0,1.0,1.0,77.0,1.0,1.0,110.0,77.0,93.5,23.33452377915607,110.0,77.0,77.0,0.0,77.0,110.0,110.0,0.0,110.0,1.3076923076923077,0.013986013986013986,110.0,1.0,77.0,1.0)
2. (LIFI LABS MANAGEMENT PTY LTD,TP-LINK TECHNOLOGIES CO.LTD.,17,75.0,2.0,194.0,1.0,1.0,77.0,1.0,1.0,117.0,77.0,97.0,28.284271247461906,117.0,77.0,77.0,0.0,77.0,117.0,117.0,0.0,117.0,2.5866666666666664,0.02666666666666667,117.0,1.0,77.0,1.0)
3. (LIFI LABS MANAGEMENT PTY LTD,TP-LINK TECHNOLOGIES CO.LTD.,17,78.0,2.0,614.0,1.0,1.0,72.0,1.0,1.0,542.0,72.0,307.0,332.3401871576773,542.0,72.0,72.0,0.0,72.0,542.0,542.0,0.0,542.0,7.871794871794871,0.02564102564102564,542.0,1.0,72.0,1.0)
4. (LIFI LABS MANAGEMENT PTY LTD,TP-LINK TECHNOLOGIES CO.LTD.,17,247.0,2.0,180.0,1.0,1.0,90.0,1.0,1.0,90.0,90.0,90.0,0.0,90.0,90.0,90.0,0.0,90.0,90.0,90.0,0.0,90.0,0.728744939271255,0.008097165991902834,90.0,1.0,90.0,1.0)
5. (LIFI LABS MANAGEMENT PTY LTD,TP-LINK TECHNOLOGIES CO.LTD.,6,120574.0,148.0,17022.0,120401.0,66.0,9933.0,120401.0,82.0,7089.0,54.0,115.01351351351352,82.3665266604211,784.0,54.0,150.49999999999997,47.48351940815483,321.0,54.0,86.45121951219512,93.01004226134614,784.0,0.14117471428334466,0.0012274619735598057,0.058878248519530565,0.0006810574663001138,0.08249931478974427,0.0005481682045830184)
6. (LIFI LABS MANAGEMENT PTY LTD,Broadcast,17,62630.0,17.0,1914.0,62630.0,17.0,1914.0,1.0,0.0,0.0,80.0,112.58823529411768,23.59835487785487,142.0,80.0,112.58823529411768,23.59835487785487,142.0,0.0,0.0,0.0,0.0,0.0305604342966629,0.0002714354143381,0.0,0.0,0.0305604342966629,0.0002714354143381)
7. (LIFI LABS MANAGEMENT PTY LTD,TP-LINK TECHNOLOGIES CO.LTD.,6,1799979.0,138.0,13458.0,1799979.0,61.0,7659.0,1754229.0,77.0,5799.0,54.0,97.52173913043475,40.89352694325176,187.0,54.0,125.55737704918036,25.61934463783434,187.0,54.0,75.31168831168834,36.95525734988607,187.0,0.0074767538954621,7.666756112154642e-05,0.003305725763284,4.3893927189665656e-05,0.0042550496422458,3.388928426387197e-05)

Examples for computer:
1. (Apple Inc.,Awair,6,2.0,2.0,132.0,1.0,1.0,78.0,1.0,1.0,54.0,54.0,66.0,16.97056274847714,78.0,78.0,78.0,0.0,78.0,54.0,54.0,0.0,54.0,66.0,1.0,54.0,1.0,78.0,1.0)
2. (Apple Inc.,Awair,6,1.0,2.0,132.0,1.0,1.0,78.0,1.0,1.0,54.0,54.0,66.0,16.97056274847714,78.0,78.0,78.0,0.0,78.0,54.0,54.0,0.0,54.0,132.0,2.0,54.0,1.0,78.0,1.0)
3. (Apple Inc.,TP-LINK TECHNOLOGIES CO.LTD.,17,68458.0,6.0,474.0,68458.0,3.0,237.0,68457.0,3.0,237.0,79.0,79.0,0.0,79.0,79.0,79.0,0.0,79.0,79.0,79.0,0.0,79.0,0.006923953372870957,8.764497940342983e-05,0.0034620272579867655,4.382312984793374e-05,0.0034619766864354786,4.382248970171492e-05)
4. (AzureWave Technology Inc.,TP-LINK TECHNOLOGIES CO.LTD.,6,804992.0,62.0,32305.0,804992.0,24.0,21889.0,804821.0,38.0,10416.0,54.0,521.0483870967743,582.9122034788508,1514.0,54.0,912.0416666666669,650.1257723914167,1404.0,54.0,274.1052631578947,368.3209374234968,1514.0,0.04013083359834632,7.701939895054857e-05,0.012942008222946468,4.7215467787247104e-05,0.027191574574654157,2.981396088408332e-05)
5. (AzureWave Technology Inc.,Broadcast,17,86223.0,147.0,13956.0,86223.0,147.0,13956.0,1.0,0.0,0.0,92.0,94.9387755102041,6.6756811666151625,110.0,92.0,94.9387755102041,6.6756811666151625,110.0,0.0,0.0,0.0,0.0,0.16185936467067952,0.0017048815281305453,0.0,0.0,0.16185936467067952,0.0017048815281305453)
6. (Apple Inc.,TP-LINK TECHNOLOGIES CO.LTD.,6,4726.0,2049.0,1669594.0,4726.0,874.0,96245.0,4721.0,1175.0,1573349.0,54.0,814.8335773548056,691.0674191830965,1514.0,54.0,110.120137299771,215.75706605529143,1514.0,54.0,1339.0204255319163,392.13855579615847,1514.0,353.27845958527297,0.4335590351248413,333.2660453293794,0.2488879474687566,20.365002115954294,0.184934405416843)
7. (AzureWave Technology Inc.,TP-LINK TECHNOLOGIES CO.LTD.,6,643.0,5.0,403.0,329.0,3.0,283.0,329.0,2.0,120.0,54.0,80.6,46.45212589322474,163.0,54.0,94.33333333333334,59.76899976855337,163.0,54.0,60.0,8.48528137423857,66.0,0.6267496111975117,0.0077760497667185,0.364741641337386,0.0060790273556231,0.8601823708206687,0.0091185410334346)

Examples for motion_sensor:
1. (Belkin International Inc.,TP-LINK TECHNOLOGIES CO.LTD.,6,219.0,3.0,426.0,219.0,2.0,280.0,1.0,1.0,146.0,66.0,142.0,74.08103670980854,214.0,66.0,140.0,104.65180361560904,214.0,146.0,146.0,0.0,146.0,1.9452054794520548,0.0136986301369863,146.0,1.0,1.278538812785388,0.0091324200913242)
2. (Belkin International Inc.,TP-LINK TECHNOLOGIES CO.LTD.,6,218.0,3.0,426.0,218.0,2.0,280.0,1.0,1.0,146.0,66.0,142.0,74.08103670980854,214.0,66.0,140.0,104.65180361560904,214.0,146.0,146.0,0.0,146.0,1.9541284403669725,0.013761467889908258,146.0,1.0,1.2844036697247707,0.009174311926605505)
3. (Belkin International Inc.,TP-LINK TECHNOLOGIES CO.LTD.,6,217.0,3.0,426.0,217.0,2.0,280.0,1.0,1.0,146.0,66.0,142.0,74.08103670980854,214.0,66.0,140.0,104.65180361560904,214.0,146.0,146.0,0.0,146.0,1.9631336405529953,0.013824884792626729,146.0,1.0,1.2903225806451613,0.009216589861751152)
4. (Belkin International Inc.,Samsung Electronics Co.Ltd,6,57.0,10.0,2032.0,56.0,5.0,524.0,53.0,5.0,1508.0,66.0,203.2,367.4817606962888,1236.0,66.0,104.8,82.36018455540274,252.0,66.0,301.6,522.3569660682242,1236.0,35.64912280701754,0.17543859649122806,28.452830188679247,0.09433962264150944,9.357142857142858,0.08928571428571429)
5. (Belkin International Inc.,IPV6 Multicast,58,119868.0,2.0,172.0,119868.0,2.0,172.0,1.0,0.0,0.0,86.0,86.0,0.0,86.0,86.0,86.0,0.0,86.0,0.0,0.0,0.0,0.0,0.0014349117362432008,1.6685020188874427e-05,0.0,0.0,0.0014349117362432008,1.6685020188874427e-05)
6. (Belkin International Inc.,TP-LINK TECHNOLOGIES CO.LTD.,6,218.0,3.0,426.0,218.0,2.0,280.0,1.0,1.0,146.0,66.0,142.0,74.08103670980854,214.0,66.0,140.0,104.65180361560904,214.0,146.0,146.0,0.0,146.0,1.9541284403669723,0.0137614678899082,146.0,1.0,1.2844036697247707,0.0091743119266055)
7. (Belkin International Inc.,Samsung Electronics Co.Ltd,6,3006.0,11.0,2106.0,3005.0,6.0,598.0,8.0,5.0,1508.0,66.0,191.45454545454544,350.7934901438063,1236.0,66.0,99.66666666666669,74.73062736700842,252.0,66.0,301.6,522.3569660682242,1236.0,0.7005988023952096,0.0036593479707252,188.5,0.625,0.1990016638935108,0.0019966722129783)

Examples for printer:
1. (Hewlett Packard,IPV6 Multicast,58,119918.0,2.0,172.0,119918.0,2.0,172.0,1.0,0.0,0.0,86.0,86.0,0.0,86.0,86.0,86.0,0.0,86.0,0.0,0.0,0.0,0.0,0.0014343134475224737,1.667806334328458e-05,0.0,0.0,0.0014343134475224737,1.667806334328458e-05)
2. (Hewlett Packard,IPV6 Multicast,58,118735.0,2.0,172.0,118735.0,2.0,172.0,1.0,0.0,0.0,86.0,86.0,0.0,86.0,86.0,86.0,0.0,86.0,0.0,0.0,0.0,0.0,0.0014486040341937928,1.6844232955741777e-05,0.0,0.0,0.0014486040341937928,1.6844232955741777e-05)
3. (Hewlett Packard,IPV6 Multicast,58,119651.0,2.0,172.0,119651.0,2.0,172.0,1.0,0.0,0.0,86.0,86.0,0.0,86.0,86.0,86.0,0.0,86.0,0.0,0.0,0.0,0.0,0.0014375141035177308,1.6715280273461986e-05,0.0,0.0,0.0014375141035177308,1.6715280273461986e-05)
4. (Hewlett Packard,Multicast,2,116386.0,2.0,92.0,116386.0,2.0,92.0,1.0,0.0,0.0,46.0,46.0,0.0,46.0,46.0,46.0,0.0,46.0,0.0,0.0,0.0,0.0,0.0007904730809547541,1.718419741205987e-05,0.0,0.0,0.0007904730809547541,1.718419741205987e-05)
5. (Hewlett Packard,TP-LINK TECHNOLOGIES CO.LTD.,6,1791976.0,50.0,5198.0,1791777.0,25.0,3500.0,1791776.0,25.0,1698.0,66.0,103.96,36.541259877970646,140.0,140.0,140.0,0.0,140.0,66.0,67.92,4.489988864128729,78.0,0.0029007084916315845,2.790215940392059e-05,0.000947663100744736,1.3952636936759951e-05,0.001953368080960968,1.3952629149721198e-05)
6. (Hewlett Packard,IPV6 Multicast,58,117569.0,2.0,172.0,117569.0,2.0,172.0,1.0,0.0,0.0,86.0,86.0,0.0,86.0,86.0,86.0,0.0,86.0,0.0,0.0,0.0,0.0,0.0014629706810468,1.7011286988917145e-05,0.0,0.0,0.0014629706810468,1.7011286988917145e-05)
7. (Hewlett Packard,Multicast,2,119986.0,2.0,92.0,119986.0,2.0,92.0,1.0,0.0,0.0,46.0,46.0,0.0,46.0,46.0,46.0,0.0,46.0,0.0,0.0,0.0,0.0,0.0007667561215475,1.666861133798943e-05,0.0,0.0,0.0007667561215475,1.666861133798943e-05)""" 
}

In [6]:
SYSTEM_DIRECTIVE_FINE_TUNING = { 
    "role": "system", 
    "content": """Given a comma-separated tuple with 30 traffic attributes of the traffic transmitted/received by different IoT devices, classify the input tuple to a device type.""" 
}

In [7]:
SYSTEM_DIRECTIVE_MAP = {
    'PROMPT_TUNING': {
        0: SYSTEM_DIRECTIVE_PROMPT_TUNING_0_EXAMPLE,
        3: SYSTEM_DIRECTIVE_PROMPT_TUNING_3_EXAMPLE,
        5: SYSTEM_DIRECTIVE_PROMPT_TUNING_5_EXAMPLE,
        7: SYSTEM_DIRECTIVE_PROMPT_TUNING_7_EXAMPLE
    },

    'FINE_TUNING': SYSTEM_DIRECTIVE_FINE_TUNING
}

In [8]:
TRAINING_INDICES_0_EXAMPLE = pd.Index( [] )
TRAINING_INDICES_1_EXAMPLE = TRAINING_INDICES_0_EXAMPLE.append( pd.Index( [ 41032, 425, 405, 48, 106, 39748, 673855, 60, 37 ] ) )
TRAINING_INDICES_2_EXAMPLE = TRAINING_INDICES_1_EXAMPLE.append( pd.Index( [ 41341, 938, 455, 111, 374, 39751, 673860, 117, 180 ] ) )
TRAINING_INDICES_3_EXAMPLE = TRAINING_INDICES_2_EXAMPLE.append( pd.Index( [ 9, 112, 574, 433, 50, 39788, 36646, 263, 265 ] ) )
TRAINING_INDICES_4_EXAMPLE = TRAINING_INDICES_3_EXAMPLE.append( pd.Index( [ 27, 246, 6, 39, 927739, 40397, 189349, 380, 40 ] ) )
TRAINING_INDICES_5_EXAMPLE = TRAINING_INDICES_4_EXAMPLE.append( pd.Index( [ 0, 17, 188530, 4211, 107, 40860, 188326, 140, 529 ] ) )
TRAINING_INDICES_7_EXAMPLE = TRAINING_INDICES_5_EXAMPLE.append( pd.Index( [ 41360, 178, 641, 367, 1243, 1246, 653, 133, 402, 54, 40551, 110286, 36508, 188570, 213, 391, 495, 1162 ] ) )

TRAINING_INDICES_MAP = {
    0: TRAINING_INDICES_0_EXAMPLE,
    1: TRAINING_INDICES_1_EXAMPLE,
    2: TRAINING_INDICES_2_EXAMPLE,
    3: TRAINING_INDICES_3_EXAMPLE,
    4: TRAINING_INDICES_4_EXAMPLE,
    5: TRAINING_INDICES_5_EXAMPLE,
    7: TRAINING_INDICES_7_EXAMPLE
}

C:\Users\Anon\AppData\Local\Temp\ipykernel_24232\1025747582.py:2: FutureWarning: The behavior of array concatenation with empty entries is deprecated. In a future version, this will no longer exclude empty items when determining the result dtype. To retain the old behavior, exclude the empty entries before the concat operation.
  TRAINING_INDICES_1_EXAMPLE = TRAINING_INDICES_0_EXAMPLE.append( pd.Index( [ 41032, 425, 405, 48, 106, 39748, 673855, 60, 37 ] ) )


In [9]:
def is_supported( mode, num_examples ):
    if mode not in SYSTEM_DIRECTIVE_MAP: return False
    if mode == 'PROMPT_TUNING' and num_examples not in SYSTEM_DIRECTIVE_MAP[mode]: return False
    if mode == 'FINE_TUNING' and num_examples not in TRAINING_INDICES_MAP: return False
    return True

In [10]:
def per_row_cost( mode, num_examples ):
    map = {
        'FINE_TUNING': 0.0017515,
        'PROMPT_TUNING': {
            0: 0.0017515,
            3: 0.01561,
            5: 0.024248
        }
    }
    
    if not is_supported( mode, num_examples ): raise Exception( f'{mode} queries with {num_examples} examples are not supported.' )

    if mode == 'FINE_TUNING': return map[mode]
    return map[mode][num_examples]

In [11]:
def get_model( mode, num_examples ):
    if not is_supported( mode, num_examples ): raise Exception( f'{mode} queries with {num_examples} examples are not supported.' ) 
    if mode == 'FINE_TUNING': return 'ft:gpt-3.5-turbo-0613:anonschool::87XPFZ74' 
    if num_examples < 3: return 'gpt-3.5-turbo'
    return 'gpt-3.5-turbo-16k'

In [12]:
def num_tokens( mode, num_examples, s ):
    enc = tiktoken.encoding_for_model( get_model( mode, num_examples ) )
    return len( enc.encode( s ) )

In [13]:
def total_cost( mode, num_examples, prompt_tokens, completion_tokens ):
    if get_model( mode, num_examples ) == 'gpt-3.5-turbo':
        cost_per_prompt_token = 0.0015
        cost_per_completion_token = 0.002
    else:
        cost_per_prompt_token = 0.003
        cost_per_completion_token = 0.004
    
    return ( ( prompt_tokens / 1000 ) * cost_per_prompt_token ) + ( ( completion_tokens / 1000 ) * cost_per_completion_token )

In [14]:
def user_directive_from_row( row ):
    return { 
        "role": "user", 
        "content": f"""Given the following comma-separated tuple containing 30 attributes of the traffic transmitted/received by this IOT device, what is the type of this device? The ordering of the attributes is the same as the ordering that I gave you in the training examples:

({row['src_oui'].replace( ',', '' )},{row['dst_oui'].replace( ',', '' )},{row['protocol']},{row['bidirectional_total_duration_ms']},{row['bidirectional_total_packets']},{row['bidirectional_total_bytes']},{row['src2dst_total_duration_ms']},{row['src2dst_total_packets']},{row['src2dst_total_bytes']},{row['dst2src_total_duration_ms']},{row['dst2src_total_packets']},{row['dst2src_total_bytes']},{row['bidirectional_min_ps']},{row['bidirectional_mean_ps']},{row['bidirectional_stdev_ps']},{row['bidirectional_max_ps']},{row['src2dst_min_ps']},{row['src2dst_mean_ps']},{row['src2dst_stdev_ps']},{row['src2dst_max_ps']},{row['dst2src_min_ps']},{row['dst2src_mean_ps']},{row['dst2src_stdev_ps']},{row['dst2src_max_ps']},{row['bidirectional_transmission_rate_byte_ms']},{row['bidirectional_transmission_rate_ms']},{row['dst2src_transmission_rate_bytes_ms']},{row['dst2src_transmission_rate_ms']},{row['src2dst_transmission_rate_bytes_ms']},{row['src2dst_transmission_rate_ms']})

Do not include any words in your response other than the device type."""
    }

In [15]:
def user_fine_tuning_directive_from_row( row ):
    return { 
        "role": "user", 
        "content": f"({row['src_oui'].replace( ',', '' )},{row['dst_oui'].replace( ',', '' )},{row['protocol']},{row['bidirectional_total_duration_ms']},{row['bidirectional_total_packets']},{row['bidirectional_total_bytes']},{row['src2dst_total_duration_ms']},{row['src2dst_total_packets']},{row['src2dst_total_bytes']},{row['dst2src_total_duration_ms']},{row['dst2src_total_packets']},{row['dst2src_total_bytes']},{row['bidirectional_min_ps']},{row['bidirectional_mean_ps']},{row['bidirectional_stdev_ps']},{row['bidirectional_max_ps']},{row['src2dst_min_ps']},{row['src2dst_mean_ps']},{row['src2dst_stdev_ps']},{row['src2dst_max_ps']},{row['dst2src_min_ps']},{row['dst2src_mean_ps']},{row['dst2src_stdev_ps']},{row['dst2src_max_ps']},{row['bidirectional_transmission_rate_byte_ms']},{row['bidirectional_transmission_rate_ms']},{row['dst2src_transmission_rate_bytes_ms']},{row['dst2src_transmission_rate_ms']},{row['src2dst_transmission_rate_bytes_ms']},{row['src2dst_transmission_rate_ms']})"
    }

In [16]:
def assistant_directive_from_row( row ):
    return { "role": "assistant", "content": base_label_from_string( row['label'] ) }

In [17]:
def gpt_completion_from_row( mode, num_examples, row ):
    if not is_supported( mode, num_examples ): raise Exception( f'{mode} queries with {num_examples} examples are not supported.' ) 
    if SLEEP_TIME > 0: sleep( SLEEP_TIME )
    
    if mode == 'FINE_TUNING': system_directive = SYSTEM_DIRECTIVE_MAP[mode]
    else: system_directive = SYSTEM_DIRECTIVE_MAP[mode][num_examples]
    
    return openai.ChatCompletion.create(
        model=get_model( mode, num_examples ),
        messages=[
            system_directive,
            user_directive_from_row( row )
        ],
        temperature=TEMPERATURE
    )

In [18]:
def get_used_index( mode, num_examples ):
    if not is_supported( mode, num_examples ): raise Exception( f'{mode} queries with {num_examples} examples are not supported.' )
    index_path = PICKLE_PATH + f'{mode}_{num_examples}_example.pkl'
    
    if not os.path.isfile( index_path ): 
        new_idx = TRAINING_INDICES_MAP[num_examples]
        pd.Series( new_idx ).to_pickle( index_path )
    
    return pd.Index( pd.read_pickle( index_path ) )

In [19]:
def extend_used_index( mode, num_examples, newly_used_indices ):
    loaded_index = get_used_index( mode, num_examples )
    loaded_index = loaded_index.append( newly_used_indices )
    pd.Series( loaded_index ).to_pickle( PICKLE_PATH + f'{mode}_{num_examples}_example.pkl' )

In [20]:
def filter_used( mode, num_examples, df ):
    used_index = get_used_index( mode, num_examples )
    return df[~df.index.isin( used_index )]

In [21]:
def filter_unused( mode, num_examples, df ):
    used_index = get_used_index( mode, num_examples )
    return df[df.index.isin( used_index )]

In [22]:
def add_predictions( mode, num_examples, df, dataset_name, outfile_path, used_index_path ):
    if not is_supported( mode, num_examples ): raise Exception( f'{mode} queries with {num_examples} examples are not supported.' )

    column_types = { 
        'entry_id': 'int64',
        'mode': 'object',
        'num_examples': 'int64',
        'dataset_name': 'object',
        'gpt_output': 'object',
        'pred': 'object',
        'actual': 'object',
        'match': 'int64'
    }
    new_predictions = pd.DataFrame( columns=column_types.keys() )
    for key in column_types:
        new_predictions[key] = new_predictions[key].astype( column_types[key] )

    if not os.path.isfile( used_index_path ): used_index = pd.Index( [] )
    else: used_index = pd.Index( pd.read_csv( used_index_path )['0'] )

    df = df[~df.index.isin( used_index )]

    print( f'Batch Size: {len( df )}' )
    
    prompt_tokens = 0
    completion_tokens = 0
    i = 0
        
    for index, row in df.iterrows():
        i += 1
        res_valid = False
        num_fails = 0
        
        while not res_valid:
            try:
                completion = gpt_completion_from_row( mode, num_examples, row )
                res_valid = True
            except Exception as e:
                print( e )
                num_fails += 1
                if num_fails >= 5: break
                sleep( SLEEP_TIME )

        if num_fails >= 5:
            break

        prompt_tokens += completion["usage"]["prompt_tokens"]
        completion_tokens += completion["usage"]["completion_tokens"]
        
        gpt_output = completion["choices"][0]["message"]["content"]
        pred = base_label_from_string( gpt_output )
        actual = base_label_from_string( row['label'] )
        match = pred_is_correct( pred, actual )
        row = [ index, mode, num_examples, dataset_name, gpt_output, pred, actual, match ]
        
        print( f'{i}. {str( row )}' )
        new_predictions.loc[len( new_predictions.index )] = row

        used_index = used_index.append( pd.Index( [ index ] ) )
    
    if not os.path.isfile( outfile_path ):
        new_predictions.to_csv( outfile_path )

    else:
        cur_predictions = pd.read_csv( outfile_path )
        predictions = pd.concat( [ cur_predictions, new_predictions ], ignore_index=True )
        predictions.reset_index()
        predictions.to_csv( outfile_path )

    pd.Series( used_index ).to_csv( used_index_path, index=False )
    
    final_cost = total_cost( mode, num_examples, prompt_tokens, completion_tokens )
    print( f'{prompt_tokens} prompt tokens used, {completion_tokens} completion tokens used' )
    print( f'Final Cost: {final_cost}' )

    return final_cost

In [23]:
def pred_is_correct( pred, actual ):
    contains = lambda x : ( x in pred ) and ( x in actual )
    terms = [ 'camera', 'router', 'mobile', 'switch', 'speaker', 'bulb', 'computer', 'sensor', 'printer' ]
    pred = pred.lower()
    out = False

    for t in terms: 
        out = out or contains( t )

    return 1 if out or ( pred == actual ) else 0

In [24]:
def base_label_from_string( s ):
    s = s.lower()
    
    if 'camera' in s: return 'camera'
    if 'router' in s: return 'router'
    if 'mobile' in s: return 'mobile'
    if 'switch' in s: return 'switch'
    if 'speaker' in s: return 'speaker'
    if 'bulb' in s: return 'bulb'
    if 'computer' in s: return 'computer'
    if 'sensor' in s: return 'motion_sensor'
    if 'printer' in s: return 'printer'
    return 'other'

In [25]:
def generate_fine_tuning_file( df, outfile_name ):
    examples = []
    for index, row in df.iterrows():
        examples.append( { "messages": [ SYSTEM_DIRECTIVE_MAP['FINE_TUNING'], user_fine_tuning_directive_from_row( row ), assistant_directive_from_row( row ) ] } )

    n = len( examples )
    with open( outfile_name, 'w' ) as f:
        for i in range( n ):
            f.write( f'{json.dumps( examples[i] )}' )
            if i < n - 1: f.write( '\n' )

In [26]:
examples = '''
Index: 41032, Input: (Insteon,Multicast,2,112030.0,4.0,240.0,112030.0,4.0,240.0,1.0,0.0,0.0,60.0,60.0,0.0,60.0,60.0,60.0,0.0,60.0,0.0,0.0,0.0,0.0,0.0021422833169686692,3.5704721949477816e-05,0.0,0.0,0.0021422833169686692,3.5704721949477816e-05)
Index: 41341, Input: (Insteon,TP-LINK TECHNOLOGIES CO.LTD.,17,216.0,2.0,340.0,1.0,1.0,78.0,1.0,1.0,262.0,78.0,170.0,130.10764773832474,262.0,78.0,78.0,0.0,78.0,262.0,262.0,0.0,262.0,1.5740740740740742,0.009259259259259259,262.0,1.0,78.0,1.0)
(New) Index: 41360, Input: (Insteon,TP-LINK TECHNOLOGIES CO.LTD.,17,1.0,1.0,274.0,1.0,1.0,274.0,1.0,0.0,0.0,274.0,274.0,0.0,274.0,274.0,274.0,0.0,274.0,0.0,0.0,0.0,0.0,274.0,1.0,0.0,0.0,274.0,1.0)
Index: 9, Input: (Samsung Electronics Co.Ltd,Multicast,2,1.0,1.0,46.0,1.0,1.0,46.0,1.0,0.0,0.0,46.0,46.0,0.0,46.0,46.0,46.0,0.0,46.0,0.0,0.0,0.0,0.0,46.0,1.0,0.0,0.0,46.0,1.0) 
Index: 27, Input: (Samsung Electronics Co.Ltd,Belkin International Inc.,17,204.0,2.0,762.0,204.0,2.0,762.0,1.0,0.0,0.0,381.0,381.0,0.0,381.0,381.0,381.0,0.0,381.0,0.0,0.0,0.0,0.0,3.735294117647059,0.00980392156862745,0.0,0.0,3.735294117647059,0.00980392156862745)
Index: 0, Input: (Netatmo,TP-LINK TECHNOLOGIES CO.LTD.,1,1313.0,4.0,392.0,1003.0,2.0,196.0,1003.0,2.0,196.0,98.0,98.0,0.0,98.0,98.0,98.0,0.0,98.0,98.0,98.0,0.0,98.0,0.2985529322162985,0.0030464584920030465,0.19541375872382852,0.0019940179461615153,0.19541375872382852,0.0019940179461615153)
(New) Index: 178: Input: (Netatmo,IPV6 Multicast,58,118439.0,2.0,172.0,118439.0,2.0,172.0,1.0,0.0,0.0,86.0,86.0,0.0,86.0,86.0,86.0,0.0,86.0,0.0,0.0,0.0,0.0,0.001452224351776,1.6886329671814184e-05,0.0,0.0,0.001452224351776,1.6886329671814184e-05)
Output: smart_camera


Index: 425, Input: (TP-LINK TECHNOLOGIES CO.LTD.,Invoxia,6,1740517.0,102.0,15258.0,1740477.0,42.0,6924.0,1740384.0,60.0,8334.0,66.0,149.5882352941177,227.36559964037056,785.0,66.0,164.85714285714292,243.02058242939128,753.0,66.0,138.90000000000003,217.19090251572047,785.0,0.008766360799693425,5.8603277072272204e-05,0.0047885983782889295,3.447515031165536e-05,0.003978219763892312,2.4131315725516626e-05) 
(New) Index: 641, Input: (TP-LINK TECHNOLOGIES CO.LTD.,Amazon Technologies Inc.,6,51.0,2.0,132.0,1.0,1.0,66.0,1.0,1.0,66.0,66.0,66.0,0.0,66.0,66.0,66.0,0.0,66.0,66.0,66.0,0.0,66.0,2.588235294117647,0.0392156862745098,66.0,1.0,66.0,1.0)
Index: 938, Input: (TP-LINK TECHNOLOGIES CO.LTD.,Samsung Electronics Co.Ltd,6,696.0,4.0,318.0,696.0,2.0,155.0,8.0,2.0,163.0,66.0,79.5,15.92691642053372,97.0,66.0,77.5,16.263455967290593,89.0,66.0,81.5,21.920310216782973,97.0,0.45689655172413796,0.005747126436781609,20.375,0.25,0.22270114942528735,0.0028735632183908046) 
Index: 112, Input: (TP-LINK TECHNOLOGIES CO.LTD.,Netatmo,17,8117.0,4.0,1376.0,81.0,3.0,1026.0,1.0,1.0,350.0,342.0,344.0,4.0,350.0,342.0,342.0,0.0,342.0,350.0,350.0,0.0,350.0,0.16952075890107182,0.0004927929037821855,350.0,1.0,12.666666666666666,0.037037037037037035) 
Index: 246, Input: (TP-LINK TECHNOLOGIES CO.LTD.,Netatmo,17,83026.0,14.0,4796.0,74753.0,13.0,4446.0,1.0,1.0,350.0,342.0,342.57142857142856,2.1380899352993974,350.0,342.0,342.0,0.0,342.0,350.0,350.0,0.0,350.0,0.05776503745814564,0.0001686218774841616,350.0,1.0,0.05947587387797145,0.00017390606397067675) 
(New) Index: 367, Input: (TP-LINK TECHNOLOGIES CO.LTD.,Netatmo,17,1797460.0,200.0,13770.0,1797460.0,100.0,6957.0,1797101.0,100.0,6813.0,43.0,68.84999999999995,56.19248595129899,394.0,43.0,69.57,59.50829752855878,394.0,43.0,68.12999999999997,52.960373026547984,250.0,0.0076608102544701,0.0001112681227955,0.0037911057864861,5.564517520161638e-05,0.0038704616514414,5.563406139775016e-05)
Index: 17, Input: (TP-LINK TECHNOLOGIES CO.LTD.,IPV6 Multicast,58,119040.0,2.0,172.0,119040.0,2.0,172.0,1.0,0.0,0.0,86.0,86.0,0.0,86.0,86.0,86.0,0.0,86.0,0.0,0.0,0.0,0.0,0.0014448924731182795,1.6801075268817204e-05,0.0,0.0,0.0014448924731182795,1.6801075268817204e-05) 
Output: router


Index: 405, Input: (Samsung Electronics Co.Ltd,TP-LINK TECHNOLOGIES CO.LTD.,6,1.0,1.0,97.0,1.0,1.0,97.0,1.0,0.0,0.0,97.0,97.0,0.0,97.0,97.0,97.0,0.0,97.0,0.0,0.0,0.0,0.0,97.0,1.0,0.0,0.0,97.0,1.0)
Index: 455, Input: (Samsung Electronics Co.Ltd,TP-LINK TECHNOLOGIES CO.LTD.,58,5770.0,4.0,336.0,5770.0,2.0,172.0,5009.0,2.0,164.0,78.0,84.0,4.000000000000001,86.0,86.0,86.0,0.0,86.0,78.0,82.0,5.656854249492381,86.0,0.058232235701906415,0.0006932409012131716,0.0327410660810541,0.0003992812936713915,0.02980935875216638,0.0003466204506065858)
Index: 574, Input: (Samsung Electronics Co.Ltd,TP-LINK TECHNOLOGIES CO.LTD.,6,143.0,4.0,314.0,143.0,2.0,157.0,1.0,2.0,157.0,66.0,78.5,14.433756729740644,91.0,66.0,78.5,17.67766952966369,91.0,66.0,78.5,17.67766952966369,91.0,2.195804195804196,0.027972027972027972,157.0,2.0,1.097902097902098,0.013986013986013986)
(New) Index: 1243, Input: (Samsung Electronics Co.Ltd,TP-LINK TECHNOLOGIES CO.LTD.,6,568.0,17.0,3524.0,568.0,8.0,1621.0,392.0,9.0,1903.0,66.0,207.29411764705884,230.13658029143323,894.0,66.0,202.625,287.6291945742444,894.0,66.0,211.44444444444449,183.01715159453715,547.0,6.204225352112676,0.0299295774647887,4.854591836734694,0.0229591836734693,2.8538732394366195,0.0140845070422535)
(New) Index: 1246, Input: (Samsung Electronics Co.Ltd,TP-LINK TECHNOLOGIES CO.LTD.,6,2496.0,24.0,5721.0,2496.0,12.0,3061.0,2494.0,12.0,2660.0,66.0,238.375,301.0856894495392,1263.0,66.0,255.0833333333333,395.1129777057635,1263.0,66.0,221.66666666666663,181.17110613166133,519.0,2.2920673076923075,0.0096153846153846,1.0665597433841218,0.0048115477145148,1.2263621794871795,0.0048076923076923)
Index: 6, Input: (AMPAK Technology Inc.,IPV6 Multicast,58,1.0,1.0,86.0,1.0,1.0,86.0,1.0,0.0,0.0,86.0,86.0,0.0,86.0,86.0,86.0,0.0,86.0,0.0,0.0,0.0,0.0,86.0,1.0,0.0,0.0,86.0,1.0)
Index: 188530, Input: (HTC Corporation,TP-LINK TECHNOLOGIES CO.LTD.,6,172.0,10.0,1202.0,172.0,6.0,764.0,169.0,4.0,438.0,66.0,120.2,74.0342263190929,246.0,66.0,127.33333333333331,91.97100992522952,246.0,66.0,109.5,45.727453460694704,149.0,6.988372093023256,0.05813953488372093,2.591715976331361,0.023668639053254437,4.441860465116279,0.03488372093023256)
Output: mobile


Index: 48, Input: (Belkin International Inc.,TP-LINK TECHNOLOGIES CO.LTD.,6,219.0,3.0,426.0,219.0,2.0,280.0,1.0,1.0,146.0,66.0,142.0,74.08103670980854,214.0,66.0,140.0,104.65180361560904,214.0,146.0,146.0,0.0,146.0,1.9452054794520548,0.0136986301369863,146.0,1.0,1.278538812785388,0.0091324200913242)
Index: 111, Input: (Belkin International Inc.,TP-LINK TECHNOLOGIES CO.LTD.,6,217.0,3.0,426.0,217.0,2.0,280.0,1.0,1.0,146.0,66.0,142.0,74.08103670980854,214.0,66.0,140.0,104.65180361560904,214.0,146.0,146.0,0.0,146.0,1.9631336405529953,0.013824884792626729,146.0,1.0,1.2903225806451613,0.009216589861751152)
Index: 433, Input: (Belkin International Inc.,Belkin International Inc.,6,226.0,13.0,1386.0,226.0,7.0,789.0,219.0,6.0,597.0,66.0,106.61538461538464,75.45698384017486,259.0,66.0,112.71428571428572,78.7204215048723,253.0,66.0,99.5,78.20421983499355,259.0,6.132743362831858,0.05752212389380531,2.7260273972602738,0.0273972602739726,3.4911504424778763,0.030973451327433628)
(New) Index: 653, Input: (Belkin International Inc.,Belkin International Inc.,6,226.0,13.0,1394.0,226.0,7.0,797.0,219.0,6.0,597.0,66.0,107.23076923076924,76.29236948100144,259.0,66.0,113.85714285714285,80.20895330620012,253.0,66.0,99.5,78.20421983499355,259.0,6.168141592920354,0.0575221238938053,2.7260273972602738,0.0273972602739726,3.5265486725663715,0.0309734513274336)
Index: 39, Input: (Belkin International Inc.,IPV6 Multicast,58,118736.0,2.0,172.0,118736.0,2.0,172.0,1.0,0.0,0.0,86.0,86.0,0.0,86.0,86.0,86.0,0.0,86.0,0.0,0.0,0.0,0.0,0.0014485918339846382,1.684409109284463e-05,0.0,0.0,0.0014485918339846382,1.684409109284463e-05)
(New) Index: 133, Input: (Belkin International Inc.,IPV6 Multicast,58,119918.0,2.0,172.0,119918.0,2.0,172.0,1.0,0.0,0.0,86.0,86.0,0.0,86.0,86.0,86.0,0.0,86.0,0.0,0.0,0.0,0.0,0.0014343134475224,1.667806334328458e-05,0.0,0.0,0.0014343134475224,1.667806334328458e-05)
Index: 4211, Input: (Belkin International Inc.,Multicast,2,118200.0,2.0,92.0,118200.0,2.0,92.0,1.0,0.0,0.0,46.0,46.0,0.0,46.0,46.0,46.0,0.0,46.0,0.0,0.0,0.0,0.0,0.00077834179357022,1.6920473773265652e-05,0.0,0.0,0.00077834179357022,1.6920473773265652e-05)
Output: smart_switch


Index: 106, Input: (Amazon Technologies Inc.,TP-LINK TECHNOLOGIES CO.LTD.,6,1995.0,20.0,7599.0,1995.0,11.0,1469.0,1742.0,9.0,6130.0,66.0,379.9500000000001,507.56420128004754,1484.0,66.0,133.54545454545456,155.40486712864796,583.0,66.0,681.1111111111111,629.3996036788641,1484.0,3.8090225563909774,0.010025062656641603,3.51894374282434,0.005166475315729047,0.7363408521303259,0.005513784461152882)
Index: 374, Input: (Amazon Technologies Inc.,TP-LINK TECHNOLOGIES CO.LTD.,6,147372.0,28.0,2187.0,147372.0,15.0,1046.0,147163.0,13.0,1141.0,54.0,78.10714285714286,40.216395482545,257.0,54.0,69.73333333333333,20.09430149024724,95.0,54.0,87.76923076923076,54.58197786533855,257.0,0.014839996742936243,0.00018999538582634422,0.007753307556926673,8.833742177041783e-05,0.007097684770512716,0.00010178324240697012)
(New) Index: 402, Input: (Amazon Technologies Inc.,TP-LINK TECHNOLOGIES CO.LTD.,6,48702.0,26.0,7627.0,2730.0,14.0,3735.0,48486.0,12.0,3892.0,54.0,293.34615384615387,470.2936055110843,1514.0,54.0,266.7857142857143,400.6951994435956,1514.0,54.0,324.33333333333337,557.6547210225947,1514.0,0.156605478214447,0.0005338589790973,0.0802705935734026,0.0002474941220146,1.3681318681318682,0.0051282051282051)
Index: 50, Input: (Amazon Technologies Inc.,TP-LINK TECHNOLOGIES CO.LTD.,17,191.0,2.0,180.0,1.0,1.0,90.0,1.0,1.0,90.0,90.0,90.0,0.0,90.0,90.0,90.0,0.0,90.0,90.0,90.0,0.0,90.0,0.9424083769633508,0.010471204188481676,90.0,1.0,90.0,1.0)
Index: 927739, Input: (Amazon Technologies Inc.,TP-LINK TECHNOLOGIES CO.LTD.,17,309224.0,14.0,1260.0,309007.0,7.0,630.0,309007.0,7.0,630.0,90.0,90.0,0.0,90.0,90.0,90.0,0.0,90.0,90.0,90.0,0.0,90.0,0.0040747160634362145,4.527462292706905e-05,0.0020387887653030515,2.265320850336724e-05,0.0020387887653030515,2.265320850336724e-05)
(New) Index: 54, Input: (Amazon Technologies Inc.,TP-LINK TECHNOLOGIES CO.LTD.,1,57500.0,20.0,1960.0,57500.0,10.0,980.0,57500.0,10.0,980.0,98.0,98.0,0.0,98.0,98.0,98.0,0.0,98.0,98.0,98.0,0.0,98.0,0.0340869565217391,0.0003478260869565,0.0170434782608695,0.0001739130434782,0.0170434782608695,0.0001739130434782) 
Index: 107, Input: (Invoxia,TP-LINK TECHNOLOGIES CO.LTD.,6,303.0,2.0,132.0,1.0,1.0,66.0,1.0,1.0,66.0,66.0,66.0,0.0,66.0,66.0,66.0,0.0,66.0,66.0,66.0,0.0,66.0,0.43564356435643564,0.006600660066006601,66.0,1.0,66.0,1.0)
Output: smart_speaker


Index: 39748, Input: (LIFI LABS MANAGEMENT PTY LTD,TP-LINK TECHNOLOGIES CO.LTD.,17,143.0,2.0,187.0,1.0,1.0,77.0,1.0,1.0,110.0,77.0,93.5,23.33452377915607,110.0,77.0,77.0,0.0,77.0,110.0,110.0,0.0,110.0,1.3076923076923077,0.013986013986013986,110.0,1.0,77.0,1.0)
Index: 39751, Input: (LIFI LABS MANAGEMENT PTY LTD,TP-LINK TECHNOLOGIES CO.LTD.,17,75.0,2.0,194.0,1.0,1.0,77.0,1.0,1.0,117.0,77.0,97.0,28.284271247461906,117.0,77.0,77.0,0.0,77.0,117.0,117.0,0.0,117.0,2.5866666666666664,0.02666666666666667,117.0,1.0,77.0,1.0)
Index: 39788, Input: (LIFI LABS MANAGEMENT PTY LTD,TP-LINK TECHNOLOGIES CO.LTD.,17,78.0,2.0,614.0,1.0,1.0,72.0,1.0,1.0,542.0,72.0,307.0,332.3401871576773,542.0,72.0,72.0,0.0,72.0,542.0,542.0,0.0,542.0,7.871794871794871,0.02564102564102564,542.0,1.0,72.0,1.0)
Index: 40397, Input: (LIFI LABS MANAGEMENT PTY LTD,TP-LINK TECHNOLOGIES CO.LTD.,17,247.0,2.0,180.0,1.0,1.0,90.0,1.0,1.0,90.0,90.0,90.0,0.0,90.0,90.0,90.0,0.0,90.0,90.0,90.0,0.0,90.0,0.728744939271255,0.008097165991902834,90.0,1.0,90.0,1.0)
(New) Index: 40551, Input: (LIFI LABS MANAGEMENT PTY LTD,Broadcast,17,62630.0,17.0,1914.0,62630.0,17.0,1914.0,1.0,0.0,0.0,80.0,112.58823529411768,23.59835487785487,142.0,80.0,112.58823529411768,23.59835487785487,142.0,0.0,0.0,0.0,0.0,0.0305604342966629,0.0002714354143381,0.0,0.0,0.0305604342966629,0.0002714354143381)
Index: 40860, Input: (LIFI LABS MANAGEMENT PTY LTD,TP-LINK TECHNOLOGIES CO.LTD.,6,120574.0,148.0,17022.0,120401.0,66.0,9933.0,120401.0,82.0,7089.0,54.0,115.01351351351352,82.3665266604211,784.0,54.0,150.49999999999997,47.48351940815483,321.0,54.0,86.45121951219512,93.01004226134614,784.0,0.14117471428334466,0.0012274619735598057,0.058878248519530565,0.0006810574663001138,0.08249931478974427,0.0005481682045830184) 
(New) Index: 110286, Input: (LIFI LABS MANAGEMENT PTY LTD,TP-LINK TECHNOLOGIES CO.LTD.,6,1799979.0,138.0,13458.0,1799979.0,61.0,7659.0,1754229.0,77.0,5799.0,54.0,97.52173913043475,40.89352694325176,187.0,54.0,125.55737704918036,25.61934463783434,187.0,54.0,75.31168831168834,36.95525734988607,187.0,0.0074767538954621,7.666756112154642e-05,0.003305725763284,4.3893927189665656e-05,0.0042550496422458,3.388928426387197e-05)
Output: smart_bulb


(New) Index: 36508, Input: (Apple Inc.,TP-LINK TECHNOLOGIES CO.LTD.,6,4726.0,2049.0,1669594.0,4726.0,874.0,96245.0,4721.0,1175.0,1573349.0,54.0,814.8335773548056,691.0674191830965,1514.0,54.0,110.120137299771,215.75706605529143,1514.0,54.0,1339.0204255319163,392.13855579615847,1514.0,353.27845958527297,0.4335590351248413,333.2660453293794,0.2488879474687566,20.365002115954294,0.184934405416843)
Index: 673855, Input: (Apple Inc.,Awair,6,2.0,2.0,132.0,1.0,1.0,78.0,1.0,1.0,54.0,54.0,66.0,16.97056274847714,78.0,78.0,78.0,0.0,78.0,54.0,54.0,0.0,54.0,66.0,1.0,54.0,1.0,78.0,1.0)
Index: 673860, Input: (Apple Inc.,Awair,6,1.0,2.0,132.0,1.0,1.0,78.0,1.0,1.0,54.0,54.0,66.0,16.97056274847714,78.0,78.0,78.0,0.0,78.0,54.0,54.0,0.0,54.0,132.0,2.0,54.0,1.0,78.0,1.0) 
Index: 36646, Input: (Apple Inc.,TP-LINK TECHNOLOGIES CO.LTD.,17,68458.0,6.0,474.0,68458.0,3.0,237.0,68457.0,3.0,237.0,79.0,79.0,0.0,79.0,79.0,79.0,0.0,79.0,79.0,79.0,0.0,79.0,0.006923953372870957,8.764497940342983e-05,0.0034620272579867655,4.382312984793374e-05,0.0034619766864354786,4.382248970171492e-05)
(New) Index: 188570, Input: (AzureWave Technology Inc.,TP-LINK TECHNOLOGIES CO.LTD.,6,643.0,5.0,403.0,329.0,3.0,283.0,329.0,2.0,120.0,54.0,80.6,46.45212589322474,163.0,54.0,94.33333333333334,59.76899976855337,163.0,54.0,60.0,8.48528137423857,66.0,0.6267496111975117,0.0077760497667185,0.364741641337386,0.0060790273556231,0.8601823708206687,0.0091185410334346)
Index: 189349, Input: (AzureWave Technology Inc.,TP-LINK TECHNOLOGIES CO.LTD.,6,804992.0,62.0,32305.0,804992.0,24.0,21889.0,804821.0,38.0,10416.0,54.0,521.0483870967743,582.9122034788508,1514.0,54.0,912.0416666666669,650.1257723914167,1404.0,54.0,274.1052631578947,368.3209374234968,1514.0,0.04013083359834632,7.701939895054857e-05,0.012942008222946468,4.7215467787247104e-05,0.027191574574654157,2.981396088408332e-05) 
Index: 188326, Input: (AzureWave Technology Inc.,Broadcast,17,86223.0,147.0,13956.0,86223.0,147.0,13956.0,1.0,0.0,0.0,92.0,94.9387755102041,6.6756811666151625,110.0,92.0,94.9387755102041,6.6756811666151625,110.0,0.0,0.0,0.0,0.0,0.16185936467067952,0.0017048815281305453,0.0,0.0,0.16185936467067952,0.0017048815281305453)
Output: computer


Index: 60, Input: (Belkin International Inc.,TP-LINK TECHNOLOGIES CO.LTD.,6,219.0,3.0,426.0,219.0,2.0,280.0,1.0,1.0,146.0,66.0,142.0,74.08103670980854,214.0,66.0,140.0,104.65180361560904,214.0,146.0,146.0,0.0,146.0,1.9452054794520548,0.0136986301369863,146.0,1.0,1.278538812785388,0.0091324200913242)
Index: 117, Input: (Belkin International Inc.,TP-LINK TECHNOLOGIES CO.LTD.,6,218.0,3.0,426.0,218.0,2.0,280.0,1.0,1.0,146.0,66.0,142.0,74.08103670980854,214.0,66.0,140.0,104.65180361560904,214.0,146.0,146.0,0.0,146.0,1.9541284403669725,0.013761467889908258,146.0,1.0,1.2844036697247707,0.009174311926605505)
(New) Index: 213, Input: (Belkin International Inc.,TP-LINK TECHNOLOGIES CO.LTD.,6,218.0,3.0,426.0,218.0,2.0,280.0,1.0,1.0,146.0,66.0,142.0,74.08103670980854,214.0,66.0,140.0,104.65180361560904,214.0,146.0,146.0,0.0,146.0,1.9541284403669723,0.0137614678899082,146.0,1.0,1.2844036697247707,0.0091743119266055)
Index: 263, Input: (Belkin International Inc.,TP-LINK TECHNOLOGIES CO.LTD.,6,217.0,3.0,426.0,217.0,2.0,280.0,1.0,1.0,146.0,66.0,142.0,74.08103670980854,214.0,66.0,140.0,104.65180361560904,214.0,146.0,146.0,0.0,146.0,1.9631336405529953,0.013824884792626729,146.0,1.0,1.2903225806451613,0.009216589861751152)
Index: 380, Input: (Belkin International Inc.,Samsung Electronics Co.Ltd,6,57.0,10.0,2032.0,56.0,5.0,524.0,53.0,5.0,1508.0,66.0,203.2,367.4817606962888,1236.0,66.0,104.8,82.36018455540274,252.0,66.0,301.6,522.3569660682242,1236.0,35.64912280701754,0.17543859649122806,28.452830188679247,0.09433962264150944,9.357142857142858,0.08928571428571429) 
(New) Index: 391, Input: (Belkin International Inc.,Samsung Electronics Co.Ltd,6,3006.0,11.0,2106.0,3005.0,6.0,598.0,8.0,5.0,1508.0,66.0,191.45454545454544,350.7934901438063,1236.0,66.0,99.66666666666669,74.73062736700842,252.0,66.0,301.6,522.3569660682242,1236.0,0.7005988023952096,0.0036593479707252,188.5,0.625,0.1990016638935108,0.0019966722129783)
Index: 140, Input: (Belkin International Inc.,IPV6 Multicast,58,119868.0,2.0,172.0,119868.0,2.0,172.0,1.0,0.0,0.0,86.0,86.0,0.0,86.0,86.0,86.0,0.0,86.0,0.0,0.0,0.0,0.0,0.0014349117362432008,1.6685020188874427e-05,0.0,0.0,0.0014349117362432008,1.6685020188874427e-05)
Output: smart_motion_sensor


Index: 37, Input: (Hewlett Packard,IPV6 Multicast,58,119918.0,2.0,172.0,119918.0,2.0,172.0,1.0,0.0,0.0,86.0,86.0,0.0,86.0,86.0,86.0,0.0,86.0,0.0,0.0,0.0,0.0,0.0014343134475224737,1.667806334328458e-05,0.0,0.0,0.0014343134475224737,1.667806334328458e-05)
Index: 180, Input: (Hewlett Packard,IPV6 Multicast,58,118735.0,2.0,172.0,118735.0,2.0,172.0,1.0,0.0,0.0,86.0,86.0,0.0,86.0,86.0,86.0,0.0,86.0,0.0,0.0,0.0,0.0,0.0014486040341937928,1.6844232955741777e-05,0.0,0.0,0.0014486040341937928,1.6844232955741777e-05)
Index: 265, Input: (Hewlett Packard,IPV6 Multicast,58,119651.0,2.0,172.0,119651.0,2.0,172.0,1.0,0.0,0.0,86.0,86.0,0.0,86.0,86.0,86.0,0.0,86.0,0.0,0.0,0.0,0.0,0.0014375141035177308,1.6715280273461986e-05,0.0,0.0,0.0014375141035177308,1.6715280273461986e-05)
(New) Index: 495, Input: (Hewlett Packard,IPV6 Multicast,58,117569.0,2.0,172.0,117569.0,2.0,172.0,1.0,0.0,0.0,86.0,86.0,0.0,86.0,86.0,86.0,0.0,86.0,0.0,0.0,0.0,0.0,0.0014629706810468,1.7011286988917145e-05,0.0,0.0,0.0014629706810468,1.7011286988917145e-05)
Index: 40, Input: (Hewlett Packard,Multicast,2,116386.0,2.0,92.0,116386.0,2.0,92.0,1.0,0.0,0.0,46.0,46.0,0.0,46.0,46.0,46.0,0.0,46.0,0.0,0.0,0.0,0.0,0.0007904730809547541,1.718419741205987e-05,0.0,0.0,0.0007904730809547541,1.718419741205987e-05)
(New) Index: 1162, Input: (Hewlett Packard,Multicast,2,119986.0,2.0,92.0,119986.0,2.0,92.0,1.0,0.0,0.0,46.0,46.0,0.0,46.0,46.0,46.0,0.0,46.0,0.0,0.0,0.0,0.0,0.0007667561215475,1.666861133798943e-05,0.0,0.0,0.0007667561215475,1.666861133798943e-05)
Index: 529, Input: (Hewlett Packard,TP-LINK TECHNOLOGIES CO.LTD.,6,1791976.0,50.0,5198.0,1791777.0,25.0,3500.0,1791776.0,25.0,1698.0,66.0,103.96,36.541259877970646,140.0,140.0,140.0,0.0,140.0,66.0,67.92,4.489988864128729,78.0,0.0029007084916315845,2.790215940392059e-05,0.000947663100744736,1.3952636936759951e-05,0.001953368080960968,1.3952629149721198e-05)
Output: smart_printer
'''